# 🤖 AI-Powered LeetCode Assistant Server (Production Version)

A **production-ready server** that connects to **OpenAI's GPT-4.1-mini** to analyze LeetCode problem screenshots and provide intelligent solutions in real-time.

## 🔑 Prerequisites

- OpenAI API key in `.env` file as `OPENAI_API_KEY`
- Internet connection for API calls

## 📋 Endpoint Details

**`POST /process-multiple-frames-stream`**

**Input:** 
- Form data with image frames (`frame_0`, `frame_1`, etc.)
- Frame count metadata

**Output:** 
- Server-Sent Events stream with real AI analysis
- Frames saved to local `frames/` directory  
- Live streaming of problem solutions and code explanations

## 🚀 Usage

Set your OpenAI API key, run the server, and send POST request with LeetCode screenshot frames to get real AI-powered analysis.

In [6]:
import nest_asyncio
nest_asyncio.apply()

from fastapi import FastAPI, Request
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse, StreamingResponse
from PIL import Image
import json
import asyncio
import io
import os
import base64
from datetime import datetime
import time
from dotenv import load_dotenv
from openai import OpenAI

# Load environment variables
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

app = FastAPI()

# Enable CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.post("/process-multiple-frames-stream")
async def simple_frame_reader_with_save(request: Request):
    """Endpoint that reads frames, saves them, converts to base64, and sends to OpenAI API with streaming response"""
    
    try:
        print("=" * 50)
        print("🔄 READING AND PROCESSING FRAMES...")
        
        # Get content type
        content_type = request.headers.get("content-type", "")
        print(f"📋 Content-Type: {content_type}")
        
        # Parse form data
        form_data = await request.form()
        print(f"📥 Form data items: {len(form_data)}")
        print(f"🔍 Form keys: {list(form_data.keys())}")
        
        # Create frames directory
        os.makedirs("frames", exist_ok=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        # Process frames and convert to base64
        saved_frames = []
        base64_images = []
        frame_count = 0
        
        for key, value in form_data.items():
            if key.startswith('frame_') and hasattr(value, 'read'):
                try:
                    # Read file data
                    file_data = await value.read()
                    print(f"📝 Processing {key}: {len(file_data)} bytes")
                    
                    # Open image
                    img = Image.open(io.BytesIO(file_data))
                    
                    # Save frame with timestamp
                    save_path = f"frames/frame_{timestamp}_{key}.png"
                    img.save(save_path)
                    
                    # Convert to base64 for OpenAI API
                    base64_img = base64.b64encode(file_data).decode('utf-8')
                    base64_images.append(base64_img)
                    
                    saved_frames.append({
                        "key": key,
                        "filename": getattr(value, 'filename', 'unknown'),
                        "size_bytes": len(file_data),
                        "image_size": f"{img.width}x{img.height}",
                        "saved_path": save_path
                    })
                    
                    frame_count += 1
                    print(f"✅ Saved {key}: {save_path} ({img.width}x{img.height})")
                    
                except Exception as frame_error:
                    print(f"❌ Error processing {key}: {frame_error}")
            
            elif key == 'frame_count':
                expected_count = str(value)
                print(f"📊 Expected frame count: {expected_count}")
        
        print(f"🎉 Successfully saved {frame_count} frames to frames/ folder!")
        print(f"🔄 Converting {len(base64_images)} images to base64 for OpenAI API...")
        
        # Now create streaming response with initial success data + real OpenAI streaming
        async def generate_stream():
            # First yield the success response
            initial_response = {
                "success": True,
                "message": f"Successfully received and saved {frame_count} frames!",
                "frames_saved": saved_frames,
                "timestamp": timestamp,
                "save_directory": "frames/",
                "streaming": True,
                "frame_count": frame_count,
                "type": "initial"
            }
            yield f"data: {json.dumps(initial_response)}\n\n"
            
            # Small delay before starting analysis
            await asyncio.sleep(0.3)
            
            try:
                # Prepare content for OpenAI API
                # Prepare content for OpenAI API
                content = [
                    {"type": "text", 
                    "text": "Analyze these LeetCode screenshots captured while scrolling. "
                    "Different frames may show different parts of the same problem (description, examples, constraints, code editor). "
                    "Combine information from all frames to provide: 1) Problem name/number 2) Complete working code solution 3) Brief explanation. "
                    "Be concise - code first, minimal explanation."
                    "Ensure not to use any imports or libraries in the code solution"}
                ]
                # Add all base64 images to the content
                for base64_img in base64_images:
                    content.append({
                        "type": "image_url", 
                        "image_url": {"url": f"data:image/png;base64,{base64_img}"}
                    })
                
                print(f"🤖 Sending {len(base64_images)} images to OpenAI API...")
                
                # Stream response from OpenAI
                stream = client.chat.completions.create(
                    model="gpt-4.1-mini",
                    messages=[
                        {
                            "role": "user",
                            "content": content
                        }
                    ],
                    temperature=0.2,
                    stream=True
                )
                
                accumulated_content = ""
                step = 0
                
                for chunk in stream:
                    # Handle content chunks
                    if chunk.choices and chunk.choices[0].delta.content is not None:
                        content_chunk = chunk.choices[0].delta.content
                        accumulated_content += content_chunk
                        step += 1
                        
                        stream_data = {
                            "type": "stream",
                            "content": content_chunk,
                            "step": step,
                            "accumulated": accumulated_content
                        }
                        yield f"data: {json.dumps(stream_data)}\n\n"
                        
                        # Small delay to make streaming visible
                        await asyncio.sleep(0.01)
                
                print("✅ OpenAI streaming completed!")
                
            except Exception as api_error:
                print(f"❌ OpenAI API Error: {api_error}")
                error_data = {
                    "type": "stream",
                    "content": f"❌ Error calling OpenAI API: {str(api_error)}\n\nUsing fallback response...\n",
                    "error": True
                }
                yield f"data: {json.dumps(error_data)}\n\n"
                
                # Fallback message
                fallback_data = {
                    "type": "stream",
                    "content": "🤖 Unable to analyze images with AI. Please check your OpenAI API key and try again.\n"
                }
                yield f"data: {json.dumps(fallback_data)}\n\n"
            
            # Final completion message
            final_data = {
                "type": "complete",
                "content": "🎯 Analysis complete!\n",
                "total_frames_processed": frame_count,
                "detected_text": f"""**Placeholder for detected text from {frame_count} frames:**

This will be replaced with actual OCR text extraction in future versions.
For now, the AI analysis above contains the problem understanding and solution.

**Technical Details:**
- Frames processed: {frame_count}
- Images sent to AI: {len(base64_images)}
- Timestamp: {timestamp}
- Save location: frames/

**Next Steps:**
- Implement OCR text extraction
- Add text preprocessing
- Enhance problem detection accuracy"""
            }
            yield f"data: {json.dumps(final_data)}\n\n"
            
            # Send the [DONE] signal that frontend is waiting for
            yield "data: [DONE]\n\n"
        
        return StreamingResponse(
            generate_stream(),
            media_type="text/event-stream",
            headers={
                "Cache-Control": "no-cache",
                "Connection": "keep-alive",
                "Content-Type": "text/event-stream",
                "Access-Control-Allow-Origin": "*",
                "Access-Control-Allow-Methods": "*",
                "Access-Control-Allow-Headers": "*"
            }
        )
        
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()
        return JSONResponse({
            "success": False,
            "error": f"Processing failed: {str(e)}"
        })

# Start server
async def start_frame_saver_server():
    import uvicorn
    try:
        print("🚀 Starting FRAME READER & OPENAI ANALYZER server on http://localhost:8000")
        print("📁 Frames will be saved to: frames/ directory")
        print("🤖 Now includes real OpenAI GPT-4.1-mini analysis!")
        print("🔑 Make sure your OPENAI_API_KEY is set in .env file")
        
        print("💡 Send your frontend request to analyze LeetCode screenshots!")
        
        config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
        server = uvicorn.Server(config)
        await server.serve()
    except Exception as e:
        print(f"❌ Server error: {e}")

await start_frame_saver_server()

🚀 Starting FRAME READER & OPENAI ANALYZER server on http://localhost:8000
📁 Frames will be saved to: frames/ directory
🤖 Now includes real OpenAI GPT-4.1-mini analysis!
🔑 Make sure your OPENAI_API_KEY is set in .env file
💡 Send your frontend request to analyze LeetCode screenshots!


INFO:     Started server process [18776]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


🔄 READING AND PROCESSING FRAMES...
📋 Content-Type: multipart/form-data; boundary=----WebKitFormBoundaryreJ2yR7s3fBdP0SW
🔄 READING AND PROCESSING FRAMES...
📋 Content-Type: multipart/form-data; boundary=----WebKitFormBoundarynx3HJyiKJR1BWIVm
📥 Form data items: 14
🔍 Form keys: ['frame_0', 'frame_1', 'frame_2', 'frame_3', 'frame_4', 'frame_5', 'frame_6', 'frame_7', 'frame_8', 'frame_9', 'frame_10', 'frame_11', 'frame_12', 'frame_count']
📝 Processing frame_0: 1015134 bytes
✅ Saved frame_0: frames/frame_20250717_201429_frame_0.png (1280x720)
📝 Processing frame_1: 994031 bytes
✅ Saved frame_1: frames/frame_20250717_201429_frame_1.png (1280x720)
📝 Processing frame_2: 984846 bytes
✅ Saved frame_2: frames/frame_20250717_201429_frame_2.png (1280x720)
📝 Processing frame_3: 988654 bytes
✅ Saved frame_3: frames/frame_20250717_201429_frame_3.png (1280x720)
📥 Form data items: 14
🔍 Form keys: ['frame_0', 'frame_1', 'frame_2', 'frame_3', 'frame_4', 'frame_5', 'frame_6', 'frame_7', 'frame_8', 'frame_9', 

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [18776]


FOR DEVELOPMENT
- explroation on homography and panorama

PLACEHOLDERS

In [ ]:
import nest_asyncio
nest_asyncio.apply()

from fastapi import FastAPI, Request
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse, StreamingResponse
import json
import asyncio
import io
import os
from datetime import datetime
import time
from PIL import Image

app = FastAPI()

# Enable CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Simple image processor - no refinements, just homography placeholder
class ImageProcessor:
    @staticmethod
    def placeholder_homography(img):
        """Placeholder for homography correction - currently returns same image"""
        # TODO: Implement homography correction here
        # This will be replaced with actual homography implementation
        return img.copy()

@app.post("/process-multiple-frames-stream")
async def process_frames_development(request: Request):
    """Simple development endpoint - test images without LLM calls"""
    
    try:
        print("=" * 50)
        print("🔄 SIMPLE DEVELOPMENT FRAME PROCESSING...")
        
        # Create directory structure: before_homography and after_homography
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        base_dir = "development"
        session_dir = os.path.join(base_dir, timestamp)
        before_dir = os.path.join(session_dir, "before_homography")
        after_dir = os.path.join(session_dir, "after_homography")
        
        os.makedirs(before_dir, exist_ok=True)
        os.makedirs(after_dir, exist_ok=True)
        
        # Parse form data
        form_data = await request.form()
        print(f"📥 Form data items: {len(form_data)}")
        
        # Lists to store all images
        all_images = []
        processing_results = []
        frame_count = 0
        
        for key, value in form_data.items():
            if key.startswith('frame_') and hasattr(value, 'read'):
                try:
                    # Read file data
                    file_data = await value.read()
                    print(f"📝 Processing {key}: {len(file_data)} bytes")
                    
                    # Open original image directly from file data
                    original_img = Image.open(io.BytesIO(file_data))
                    
                    print(f"✅ Processing {key}")
                    
                    # Apply placeholder homography (currently does nothing)
                    processed_img = ImageProcessor.placeholder_homography(original_img)
                    
                    # Add directly to images list
                    image_data = {
                        "key": key,
                        "original_image": original_img,
                        "processed_image": processed_img,
                        "file_data": file_data,
                        "filename": getattr(value, 'filename', 'unknown'),
                        "size_bytes": len(file_data),
                        "image_size": f"{original_img.width}x{original_img.height}"
                    }
                    all_images.append(image_data)
                    
                    # Save both images
                    before_path = os.path.join(before_dir, f"{key}_original.png")
                    after_path = os.path.join(after_dir, f"{key}_homography.png")
                    
                    original_img.save(before_path)
                    processed_img.save(after_path)
                    
                    processing_results.append({
                        "frame_key": key,
                        "original_path": before_path,
                        "processed_path": after_path,
                        "original_size": f"{original_img.width}x{original_img.height}",
                        "original_file_size": len(file_data),
                        "processed_file_size": os.path.getsize(after_path),
                        "status": "processed"
                    })
                    
                    frame_count += 1
                    print(f"✅ Processed {key} - saved to directories")
                    
                except Exception as frame_error:
                    print(f"❌ Error processing {key}: {frame_error}")
        
        print(f"🎉 Successfully processed {frame_count} frames!")
        print(f"📋 All {len(all_images)} images loaded into list for testing")
        
        # Create streaming response
        async def generate_development_stream():
            # Initial response
            initial_response = {
                "success": True,
                "message": f"Development processing complete! {frame_count} images processed",
                "timestamp": timestamp,
                "session_directory": session_dir,
                "directories": {
                    "session": session_dir,
                    "before_homography": before_dir,
                    "after_homography": after_dir
                },
                "frame_count": frame_count,
                "all_images_count": len(all_images),
                "type": "initial"
            }
            yield f"data: {json.dumps(initial_response)}\n\n"
            
            await asyncio.sleep(0.3)
            
            # Stream processing results
            for i, result in enumerate(processing_results):
                stream_data = {
                    "type": "frame_result",
                    "frame_index": i + 1,
                    "total_frames": frame_count,
                    "result": result
                }
                yield f"data: {json.dumps(stream_data)}\n\n"
                await asyncio.sleep(0.1)
            
            # Summary
            total_original_size = sum([img["size_bytes"] for img in all_images])
            
            summary_data = {
                "type": "summary",
                "content": f"""🔬 **Simple Development Processing Summary**

📊 **Frame Statistics:**
- Total frames processed: {frame_count}
- All frames accepted: {frame_count}

📁 **Total Size:** {total_original_size:,} bytes
📍 **Session Directory Structure:**
   - Main session: {session_dir}
   - Before homography: {before_dir}
   - After homography: {after_dir}

🖼️ **Images Loaded into Memory:**
- Total images: {len(all_images)}
- Each image contains: PIL Image objects, metadata
- Ready for testing and development

🔧 **Simple Processing Flow:**
1. Read file data from form ✅
2. Create PIL Image directly from bytes ✅
3. Apply placeholder homography (no-op) ✅
4. Add to all_images list ✅
5. Save to organized directory structure ✅

📂 **Directory Organization:**
development/
└── {timestamp}/
    ├── before_homography/
    └── after_homography/

🔍 **Development Ready:**
- All {len(all_images)} images available in `all_images` list
- No image processing applied yet
- No LLM calls - pure testing environment
- Ready for homography implementation

💡 **Next Steps:**
- Implement actual homography correction
- Add any additional processing as needed
- Test with real image transformations

🚀 **Simple, clean development pipeline!**""",
                "total_original_size": total_original_size,
                "images_in_memory": len(all_images),
                "processing_results": processing_results
            }
            yield f"data: {json.dumps(summary_data)}\n\n"
            
            yield "data: [DONE]\n\n"
        
        return StreamingResponse(
            generate_development_stream(),
            media_type="text/event-stream",
            headers={
                "Cache-Control": "no-cache",
                "Connection": "keep-alive",
                "Content-Type": "text/event-stream",
                "Access-Control-Allow-Origin": "*",
                "Access-Control-Allow-Methods": "*",
                "Access-Control-Allow-Headers": "*"
            }
        )
        
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()
        return JSONResponse({
            "success": False,
            "error": f"Development processing failed: {str(e)}"
        })

# Start server
async def start_development_server():
    import uvicorn
    try:
        print("🚀 Starting SIMPLE DEVELOPMENT server on http://localhost:8000")
        print("📁 Images organized: before_homography → after_homography")
        print("🔧 Homography placeholder ready for implementation")
        print("📂 Directory structure: development/TIMESTAMP/[before_homography|after_homography]/")
        print("📋 Perfect for testing without processing overhead")
        print("💰 NO LLM calls - pure development environment!")
        
        config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
        server = uvicorn.Server(config)
        await server.serve()
    except Exception as e:
        print(f"❌ Server error: {e}")

await start_development_server()

🚀 Starting SIMPLE DEVELOPMENT server on http://localhost:8000
📁 Images organized: before_homography → after_homography
🔧 Homography placeholder ready for implementation
📂 Directory structure: development/TIMESTAMP/[before_homography|after_homography]/
📋 Perfect for testing without processing overhead
💰 NO LLM calls - pure development environment!


INFO:     Started server process [28784]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


🔄 SIMPLE DEVELOPMENT FRAME PROCESSING...
🔄 SIMPLE DEVELOPMENT FRAME PROCESSING...
📥 Form data items: 7
📝 Processing frame_0: 1697426 bytes
✅ Processing frame_0
✅ Processed frame_0 - saved to directories
📝 Processing frame_1: 1754668 bytes
✅ Processing frame_1
✅ Processed frame_1 - saved to directories
📝 Processing frame_2: 1732274 bytes
✅ Processing frame_2
✅ Processed frame_2 - saved to directories
📝 Processing frame_3: 1654518 bytes
✅ Processing frame_3
✅ Processed frame_3 - saved to directories
📝 Processing frame_4: 1727742 bytes
✅ Processing frame_4
✅ Processed frame_4 - saved to directories
📝 Processing frame_5: 1894885 bytes
✅ Processing frame_5
✅ Processed frame_5 - saved to directories
🎉 Successfully processed 6 frames!
📋 All 6 images loaded into list for testing
INFO:     127.0.0.1:55588 - "POST /process-multiple-frames-stream HTTP/1.1" 200 OK
📥 Form data items: 7
📝 Processing frame_0: 1697426 bytes
✅ Processing frame_0
✅ Processed frame_0 - saved to directories
📝 Processing f

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [28784]


PANORAMA NOT WOKRING BECAUSE OF HEADER AND FOOTER

In [12]:
import nest_asyncio
nest_asyncio.apply()

from fastapi import FastAPI, Request
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse, StreamingResponse
import json
import asyncio
import io
import os
from datetime import datetime
import time
from PIL import Image
import cv2
import numpy as np

app = FastAPI()

# Enable CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Image processor with enhanced homography and panorama functionality
class ImageProcessor:
    @staticmethod
    def order_points(pts):
        """Order points in the order: top-left, top-right, bottom-right, bottom-left"""
        rect = np.zeros((4, 2), dtype="float32")
        
        # Sum and difference to find corners
        s = pts.sum(axis=1)
        diff = np.diff(pts, axis=1)
        
        rect[0] = pts[np.argmin(s)]      # top-left
        rect[2] = pts[np.argmax(s)]      # bottom-right
        rect[1] = pts[np.argmin(diff)]   # top-right
        rect[3] = pts[np.argmax(diff)]   # bottom-left
        
        return rect
    
    @staticmethod
    def four_point_transform(image, pts):
        """Apply perspective transform to get bird's eye view"""
        rect = ImageProcessor.order_points(pts)
        (tl, tr, br, bl) = rect
        
        # Compute width and height of new image
        widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
        widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
        maxWidth = max(int(widthA), int(widthB))
        
        heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
        heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
        maxHeight = max(int(heightA), int(heightB))
        
        # Construct destination points
        dst = np.array([
            [0, 0],
            [maxWidth - 1, 0],
            [maxWidth - 1, maxHeight - 1],
            [0, maxHeight - 1]
        ], dtype="float32")
        
        # Compute perspective transform matrix and apply it
        M = cv2.getPerspectiveTransform(rect, dst)
        warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
        
        return warped
    
    @staticmethod
    def document_scanner_homography(img, save_debug=True, debug_dir=None, frame_key="unknown"):
        """Apply document scanner homography transformation with quality checks and rejection"""
        try:
            # Convert PIL to OpenCV
            img_cv = np.array(img)
            if len(img_cv.shape) == 3:
                img_cv = cv2.cvtColor(img_cv, cv2.COLOR_RGB2BGR)
            
            original = img_cv.copy()
            gray = cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY)
            
            # Apply Gaussian blur
            blurred = cv2.GaussianBlur(gray, (5, 5), 0)
            
            # Edge detection
            edged = cv2.Canny(blurred, 75, 200)
            
            # Save debug images if requested
            if save_debug and debug_dir:
                debug_frame_dir = os.path.join(debug_dir, frame_key)
                os.makedirs(debug_frame_dir, exist_ok=True)
                
                # Save processing steps
                cv2.imwrite(os.path.join(debug_frame_dir, "01_grayscale.png"), gray)
                cv2.imwrite(os.path.join(debug_frame_dir, "02_blurred.png"), blurred)
                cv2.imwrite(os.path.join(debug_frame_dir, "03_edges.png"), edged)
            
            # Find contours
            contours, _ = cv2.findContours(edged, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
            contours = sorted(contours, key=cv2.contourArea, reverse=True)[:5]
            
            print(f"🔍 Found {len(contours)} contours for {frame_key}")
            
            # Create debug image with all contours
            debug_contours = original.copy()
            cv2.drawContours(debug_contours, contours, -1, (0, 255, 0), 2)
            
            # Find the largest contour with 4 points (document)
            screenCnt = None
            contour_info = []
            rejection_reason = None
            
            for idx, c in enumerate(contours):
                area = cv2.contourArea(c)
                peri = cv2.arcLength(c, True)
                approx = cv2.approxPolyDP(c, 0.02 * peri, True)
                
                contour_info.append({
                    "index": idx,
                    "area": area,
                    "perimeter": peri,
                    "points": len(approx)
                })
                
                print(f"  Contour {idx}: Area={area:.0f}, Perimeter={peri:.0f}, Points={len(approx)}")
                
                # Draw contour number on debug image
                if len(approx) >= 3:
                    M = cv2.moments(c)
                    if M["m00"] != 0:
                        cx = int(M["m10"] / M["m00"])
                        cy = int(M["m01"] / M["m00"])
                        cv2.putText(debug_contours, f"{idx}({len(approx)}p)", (cx-20, cy), 
                                  cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
                
                if len(approx) == 4:
                    # QUALITY CHECKS FOR 4-POINT CONTOUR
                    
                    # 1. Area check - contour should be reasonably large
                    min_area = (img_cv.shape[0] * img_cv.shape[1]) * 0.1  # At least 10% of image
                    if area < min_area:
                        rejection_reason = f"contour_too_small_area_{area:.0f}_min_{min_area:.0f}"
                        print(f"  ❌ Contour {idx} rejected: too small (area: {area:.0f} < {min_area:.0f})")
                        continue
                    
                    # 2. Aspect ratio check - should look reasonable
                    rect = ImageProcessor.order_points(approx.reshape(4, 2))
                    (tl, tr, br, bl) = rect
                    
                    width1 = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
                    width2 = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
                    height1 = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
                    height2 = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
                    
                    avg_width = (width1 + width2) / 2
                    avg_height = (height1 + height2) / 2
                    aspect_ratio = max(avg_width, avg_height) / min(avg_width, avg_height)
                    
                    if aspect_ratio > 5.0:  # Too extreme aspect ratio
                        rejection_reason = f"extreme_aspect_ratio_{aspect_ratio:.2f}"
                        print(f"  ❌ Contour {idx} rejected: extreme aspect ratio ({aspect_ratio:.2f})")
                        continue
                    
                    # 3. Corner angle check - corners should be roughly 90 degrees
                    def angle_between_vectors(v1, v2):
                        cos_angle = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
                        cos_angle = np.clip(cos_angle, -1.0, 1.0)
                        return np.degrees(np.arccos(cos_angle))
                    
                    corners = approx.reshape(4, 2)
                    angles = []
                    for i in range(4):
                        p1 = corners[i]
                        p2 = corners[(i + 1) % 4]
                        p3 = corners[(i + 2) % 4]
                        
                        v1 = p1 - p2
                        v2 = p3 - p2
                        angle = angle_between_vectors(v1, v2)
                        angles.append(angle)
                    
                    # Check if any angle is too far from 90 degrees
                    angle_threshold = 45  # degrees deviation from 90
                    bad_angles = [abs(angle - 90) for angle in angles if abs(angle - 90) > angle_threshold]
                    
                    if bad_angles:
                        rejection_reason = f"bad_corner_angles_{bad_angles}"
                        print(f"  ❌ Contour {idx} rejected: bad corner angles {angles}")
                        continue
                    
                    print(f"  ✅ Found valid 4-point contour at index {idx}!")
                    print(f"    Area: {area:.0f}, Aspect ratio: {aspect_ratio:.2f}, Angles: {[f'{a:.1f}°' for a in angles]}")
                    
                    screenCnt = approx
                    
                    # Draw the selected contour in red
                    cv2.drawContours(debug_contours, [approx], -1, (0, 0, 255), 3)
                    
                    # Draw corner points
                    for i, point in enumerate(approx):
                        cv2.circle(debug_contours, tuple(point[0]), 8, (255, 255, 0), -1)
                        cv2.putText(debug_contours, str(i), tuple(point[0] + 10), 
                                  cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
                    break
            
            # Save debug images
            if save_debug and debug_dir:
                cv2.imwrite(os.path.join(debug_frame_dir, "04_contours_detected.png"), debug_contours)
                
                # Save contour info as text
                with open(os.path.join(debug_frame_dir, "contour_info.txt"), "w") as f:
                    f.write(f"Contour Analysis for {frame_key}\n")
                    f.write("=" * 40 + "\n")
                    for info in contour_info:
                        f.write(f"Contour {info['index']}: Area={info['area']:.0f}, "
                               f"Perimeter={info['perimeter']:.0f}, Points={info['points']}\n")
                    f.write(f"\nSelected contour: {'Found' if screenCnt is not None else 'None'}\n")
                    if rejection_reason:
                        f.write(f"Rejection reason: {rejection_reason}\n")
            
            if screenCnt is not None:
                print(f"✅ Applying homography transformation for {frame_key}")
                
                # Apply the four point transform
                warped = ImageProcessor.four_point_transform(original, screenCnt.reshape(4, 2))
                
                # POST-WARP QUALITY CHECKS
                
                # 1. Check if warped image is too small
                min_warped_size = 200  # pixels
                if warped.shape[0] < min_warped_size or warped.shape[1] < min_warped_size:
                    rejection_reason = f"warped_too_small_{warped.shape[1]}x{warped.shape[0]}"
                    print(f"  ❌ Warped result rejected: too small ({warped.shape[1]}x{warped.shape[0]})")
                    raise ValueError(f"Warped image too small: {warped.shape}")
                
                # 2. Check if warped image is extremely distorted
                warp_aspect = max(warped.shape[0], warped.shape[1]) / min(warped.shape[0], warped.shape[1])
                if warp_aspect > 10.0:
                    rejection_reason = f"warped_extreme_aspect_{warp_aspect:.2f}"
                    print(f"  ❌ Warped result rejected: extreme aspect ratio ({warp_aspect:.2f})")
                    raise ValueError(f"Warped image too distorted: aspect ratio {warp_aspect:.2f}")
                
                # 3. Check if warped image is mostly black/empty
                gray_warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
                non_zero_pixels = np.count_nonzero(gray_warped > 50)  # Pixels brighter than 50
                total_pixels = gray_warped.shape[0] * gray_warped.shape[1]
                content_ratio = non_zero_pixels / total_pixels
                
                if content_ratio < 0.3:  # Less than 30% content
                    rejection_reason = f"warped_mostly_empty_{content_ratio:.2f}"
                    print(f"  ❌ Warped result rejected: mostly empty ({content_ratio:.2%} content)")
                    raise ValueError(f"Warped image mostly empty: {content_ratio:.2%} content")
                
                # Save warped result
                if save_debug and debug_dir:
                    cv2.imwrite(os.path.join(debug_frame_dir, "05_warped_result.png"), warped)
                    
                    # Save success info
                    with open(os.path.join(debug_frame_dir, "success_info.txt"), "w") as f:
                        f.write(f"Homography successful for {frame_key}\n")
                        f.write(f"Warped size: {warped.shape[1]}x{warped.shape[0]}\n")
                        f.write(f"Warped aspect ratio: {warp_aspect:.2f}\n")
                        f.write(f"Content ratio: {content_ratio:.2%}\n")
                
                # Convert back to RGB for PIL
                if len(warped.shape) == 3:
                    warped = cv2.cvtColor(warped, cv2.COLOR_BGR2RGB)
                
                return Image.fromarray(warped), "success"
            else:
                if not rejection_reason:
                    rejection_reason = "no_4_point_contour_found"
                
                print(f"⚠️ No valid 4-point contour found for {frame_key}")
                
                # Save info about why it failed
                if save_debug and debug_dir:
                    with open(os.path.join(debug_frame_dir, "failure_reason.txt"), "w") as f:
                        f.write(f"Homography failed for {frame_key}\n")
                        f.write(f"Reason: {rejection_reason}\n\n")
                        f.write("Suggestions:\n")
                        f.write("1. Check if image has clear document edges\n")
                        f.write("2. Adjust Canny edge detection parameters\n")
                        f.write("3. Try different contour approximation epsilon\n")
                        f.write("4. Ensure good contrast between document and background\n")
                        f.write("5. Make sure document occupies significant portion of image\n")
                
                return None, rejection_reason
                
        except Exception as e:
            error_msg = f"error_{str(e).replace(' ', '_')}"
            print(f"❌ Homography error for {frame_key}: {e}")
            
            if save_debug and debug_dir:
                debug_frame_dir = os.path.join(debug_dir, frame_key)
                os.makedirs(debug_frame_dir, exist_ok=True)
                with open(os.path.join(debug_frame_dir, "error.txt"), "w") as f:
                    f.write(f"Error processing {frame_key}: {str(e)}\n")
            
            return None, error_msg
    
    @staticmethod
    def create_panorama(images):
        """Create panorama from multiple images using SIFT features"""
        try:
            if len(images) < 2:
                return images[0] if images else None
            
            # Convert PIL images to OpenCV format
            cv_images = []
            for img in images:
                img_cv = np.array(img)
                if len(img_cv.shape) == 3:
                    img_cv = cv2.cvtColor(img_cv, cv2.COLOR_RGB2BGR)
                cv_images.append(img_cv)
            
            # Create SIFT detector
            sift = cv2.SIFT_create()
            
            # Start with the first image
            result = cv_images[0]
            
            for i in range(1, len(cv_images)):
                # Find keypoints and descriptors
                kp1, des1 = sift.detectAndCompute(result, None)
                kp2, des2 = sift.detectAndCompute(cv_images[i], None)
                
                if des1 is None or des2 is None:
                    print(f"⚠️ No features found in image {i}, skipping...")
                    continue
                
                # Match features
                bf = cv2.BFMatcher()
                matches = bf.knnMatch(des1, des2, k=2)
                
                # Apply ratio test
                good_matches = []
                for match_pair in matches:
                    if len(match_pair) == 2:
                        m, n = match_pair
                        if m.distance < 0.75 * n.distance:
                            good_matches.append(m)
                
                if len(good_matches) > 10:
                    # Extract matched points
                    src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
                    dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
                    
                    # Find homography
                    M, mask = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)
                    
                    if M is not None:
                        # Get dimensions
                        h1, w1 = result.shape[:2]
                        h2, w2 = cv_images[i].shape[:2]
                        
                        # Transform corners of second image
                        corners = np.float32([[0, 0], [w2, 0], [w2, h2], [0, h2]]).reshape(-1, 1, 2)
                        transformed_corners = cv2.perspectiveTransform(corners, M)
                        
                        # Calculate output size
                        all_corners = np.concatenate([
                            np.float32([[0, 0], [w1, 0], [w1, h1], [0, h1]]).reshape(-1, 1, 2),
                            transformed_corners
                        ])
                        
                        [x_min, y_min] = np.int32(all_corners.min(axis=0).ravel() - 0.5)
                        [x_max, y_max] = np.int32(all_corners.max(axis=0).ravel() + 0.5)
                        
                        # Translation matrix
                        translation = np.array([[1, 0, -x_min], [0, 1, -y_min], [0, 0, 1]], dtype=np.float32)
                        
                        # Warp second image
                        warped = cv2.warpPerspective(cv_images[i], translation.dot(M), (x_max - x_min, y_max - y_min))
                        
                        # Place first image
                        result_warped = cv2.warpPerspective(result, translation, (x_max - x_min, y_max - y_min))
                        
                        # Blend images
                        mask = (warped > 0).astype(np.uint8)
                        result = result_warped * (1 - mask) + warped * mask
                        result = result.astype(np.uint8)
                    else:
                        print(f"⚠️ Could not find homography for image {i}")
                else:
                    print(f"⚠️ Not enough good matches for image {i}")
            
            # Convert back to RGB for PIL
            if len(result.shape) == 3:
                result = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)
            
            return Image.fromarray(result)
            
        except Exception as e:
            print(f"❌ Panorama error: {e}")
            return images[0] if images else None

@app.post("/process-multiple-frames-stream")
async def process_frames_development(request: Request):
    """Development endpoint with homography and panorama functionality with frame rejection"""
    
    try:
        print("=" * 50)
        print("🔄 DEVELOPMENT FRAME PROCESSING WITH HOMOGRAPHY & PANORAMA & REJECTION...")
        
        # Create directory structure
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        base_dir = "development"
        session_dir = os.path.join(base_dir, timestamp)
        before_dir = os.path.join(session_dir, "before_homography")
        after_dir = os.path.join(session_dir, "after_homography")
        rejected_dir = os.path.join(session_dir, "rejected_homography")
        panorama_dir = os.path.join(session_dir, "panorama")
        debug_dir = os.path.join(session_dir, "debug_visualization")
        
        os.makedirs(before_dir, exist_ok=True)
        os.makedirs(after_dir, exist_ok=True)
        os.makedirs(rejected_dir, exist_ok=True)
        os.makedirs(panorama_dir, exist_ok=True)
        os.makedirs(debug_dir, exist_ok=True)
        
        # Parse form data
        form_data = await request.form()
        print(f"📥 Form data items: {len(form_data)}")
        
        # Lists to store all images
        all_images = []
        rejected_images = []
        processing_results = []
        frame_count = 0
        rejected_count = 0
        
        for key, value in form_data.items():
            if key.startswith('frame_') and hasattr(value, 'read'):
                try:
                    # Read file data
                    file_data = await value.read()
                    print(f"📝 Processing {key}: {len(file_data)} bytes")
                    
                    # Open original image directly from file data
                    original_img = Image.open(io.BytesIO(file_data))
                    
                    print(f"✅ Processing {key}")
                    
                    # Apply document scanner homography with quality checks
                    processed_img, status = ImageProcessor.document_scanner_homography(
                        original_img, 
                        save_debug=True, 
                        debug_dir=debug_dir, 
                        frame_key=key
                    )
                    
                    if processed_img is not None and status == "success":
                        # SUCCESS - Add to accepted processing results
                        image_data = {
                            "key": key,
                            "original_image": original_img,
                            "processed_image": processed_img,
                            "file_data": file_data,
                            "filename": getattr(value, 'filename', 'unknown'),
                            "size_bytes": len(file_data),
                            "image_size": f"{original_img.width}x{original_img.height}",
                            "status": "accepted"
                        }
                        all_images.append(image_data)
                        
                        # Save both images
                        before_path = os.path.join(before_dir, f"{key}_original.png")
                        after_path = os.path.join(after_dir, f"{key}_homography.png")
                        
                        original_img.save(before_path)
                        processed_img.save(after_path)
                        
                        processing_results.append({
                            "frame_key": key,
                            "original_path": before_path,
                            "processed_path": after_path,
                            "debug_path": os.path.join(debug_dir, key),
                            "original_size": f"{original_img.width}x{original_img.height}",
                            "processed_size": f"{processed_img.width}x{processed_img.height}",
                            "original_file_size": len(file_data),
                            "processed_file_size": os.path.getsize(after_path),
                            "status": "accepted"
                        })
                        
                        frame_count += 1
                        print(f"✅ ACCEPTED {key} - homography successful")
                    else:
                        # REJECTED - Save to rejected folder
                        rejected_path = os.path.join(rejected_dir, f"{key}_rejected_{status}.png")
                        original_img.save(rejected_path)
                        
                        rejected_images.append({
                            "key": key,
                            "filename": getattr(value, 'filename', 'unknown'),
                            "size_bytes": len(file_data),
                            "image_size": f"{original_img.width}x{original_img.height}",
                            "rejection_reason": status,
                            "rejected_path": rejected_path,
                            "debug_path": os.path.join(debug_dir, key)
                        })
                        
                        rejected_count += 1
                        print(f"❌ REJECTED {key} - {status}")
                        
                except Exception as frame_error:
                    print(f"❌ Error processing {key}: {frame_error}")
        
        # Create panorama from accepted processed images
        panorama_img = None
        panorama_path = None
        if len(all_images) > 1:
            print("🔄 Creating panorama from accepted processed images...")
            processed_images = [img_data["processed_image"] for img_data in all_images]
            panorama_img = ImageProcessor.create_panorama(processed_images)
            
            if panorama_img:
                panorama_path = os.path.join(panorama_dir, f"panorama_{timestamp}.png")
                panorama_img.save(panorama_path)
                print(f"✅ Panorama created: {panorama_path}")
        
        print(f"🎉 Successfully processed {frame_count} frames!")
        print(f"❌ Rejected {rejected_count} frames!")
        print(f"📋 {len(all_images)} accepted images loaded into list for testing")
        
        # Create streaming response
        async def generate_development_stream():
            # Initial response
            initial_response = {
                "success": True,
                "message": f"Development processing complete! {frame_count} accepted, {rejected_count} rejected",
                "timestamp": timestamp,
                "session_directory": session_dir,
                "directories": {
                    "session": session_dir,
                    "before_homography": before_dir,
                    "after_homography": after_dir,
                    "rejected_homography": rejected_dir,
                    "panorama": panorama_dir,
                    "debug_visualization": debug_dir
                },
                "frame_count": frame_count,
                "rejected_count": rejected_count,
                "all_images_count": len(all_images),
                "panorama_created": panorama_img is not None,
                "panorama_path": panorama_path,
                "type": "initial"
            }
            yield f"data: {json.dumps(initial_response)}\n\n"
            
            await asyncio.sleep(0.3)
            
            # Stream processing results for accepted frames
            for i, result in enumerate(processing_results):
                stream_data = {
                    "type": "frame_result",
                    "frame_index": i + 1,
                    "total_frames": frame_count,
                    "result": result
                }
                yield f"data: {json.dumps(stream_data)}\n\n"
                await asyncio.sleep(0.1)
            
            # Stream rejected frames info
            for i, rejected in enumerate(rejected_images):
                stream_data = {
                    "type": "rejected_frame",
                    "rejected_index": i + 1,
                    "total_rejected": rejected_count,
                    "result": rejected
                }
                yield f"data: {json.dumps(stream_data)}\n\n"
                await asyncio.sleep(0.1)
            
            # Summary
            total_original_size = sum([img["size_bytes"] for img in all_images])
            total_rejected_size = sum([img["size_bytes"] for img in rejected_images])
            
            summary_data = {
                "type": "summary",
                "content": f"""🔬 **Advanced Development Processing with Frame Rejection**

📊 **Frame Statistics:**
- ✅ Total frames accepted: {frame_count}
- ❌ Total frames rejected: {rejected_count}
- 📈 Acceptance rate: {(frame_count/(frame_count+rejected_count)*100):.1f}%
- 🖼️ Panorama created: {"✅ Yes" if panorama_img else "❌ No"}

📁 **Data Size:**
- Accepted frames: {total_original_size:,} bytes
- Rejected frames: {total_rejected_size:,} bytes
- Total processed: {(total_original_size + total_rejected_size):,} bytes

📍 **Session Directory Structure:**
   - Main session: {session_dir}
   - ✅ Before homography: {before_dir}
   - ✅ After homography: {after_dir}
   - ❌ **Rejected frames: {rejected_dir}**
   - 🖼️ Panorama: {panorama_dir}
   - 🔍 **Debug visualization: {debug_dir}**

🖼️ **Images in Memory:**
- Accepted images: {len(all_images)}
- Each image contains: PIL Image objects, metadata
- Ready for testing and development

🔧 **Enhanced Processing Flow:**
1. Read file data from form ✅
2. Create PIL Image directly from bytes ✅
3. Apply document scanner homography with quality checks ✅
4. **🔍 Quality validation (area, aspect ratio, corner angles) ✅**
5. **📊 Post-warp validation (size, distortion, content) ✅**
6. **❌ Reject poor quality transforms ✅**
7. **🔍 Save debug visualization images ✅**
8. Create panorama from ACCEPTED images only ✅
9. Add to organized directory structure ✅

📂 **Directory Organization:**
development/
└── {timestamp}/
    ├── before_homography/ (accepted only)
    ├── after_homography/ (accepted only)
    ├── **rejected_homography/** ❌
    ├── panorama/
    └── **debug_visualization/**
        └── **frame_X/**
            ├── **01_grayscale.png**
            ├── **02_blurred.png**
            ├── **03_edges.png**
            ├── **04_contours_detected.png** (with boxes!)
            ├── **05_warped_result.png** (if successful)
            ├── **success_info.txt** (if successful)
            ├── **failure_reason.txt** (if failed)
            └── **contour_info.txt**

🔍 **Quality Checks Implemented:**
- ✅ **Pre-warp validation:**
  - Minimum contour area (10% of image)
  - Reasonable aspect ratio (< 5:1)
  - Corner angles close to 90° (±45°)
- ✅ **Post-warp validation:**
  - Minimum warped size (200px)
  - Maximum aspect ratio (< 10:1)
  - Content density (> 30% non-black pixels)

❌ **Rejection Reasons Tracked:**
- contour_too_small_area_X_min_Y
- extreme_aspect_ratio_X
- bad_corner_angles_[list]
- warped_too_small_WxH
- warped_extreme_aspect_X
- warped_mostly_empty_X%
- no_4_point_contour_found
- error_[exception_details]

💡 **Debug Features:**
- **Check {debug_dir} for detailed analysis!**
- Each frame has complete processing visualization
- Success/failure reasons clearly documented
- Quality metrics saved for analysis
- Rejected frames preserved for inspection

🚀 **Production-ready homography pipeline with quality control!**""",
                "total_original_size": total_original_size,
                "total_rejected_size": total_rejected_size,
                "accepted_images_in_memory": len(all_images),
                "rejected_images_count": len(rejected_images),
                "processing_results": processing_results,
                "rejected_results": rejected_images,
                "panorama_info": {
                    "created": panorama_img is not None,
                    "path": panorama_path,
                    "size": f"{panorama_img.width}x{panorama_img.height}" if panorama_img else None
                },
                "quality_metrics": {
                    "acceptance_rate": f"{(frame_count/(frame_count+rejected_count)*100):.1f}%" if (frame_count+rejected_count) > 0 else "N/A",
                    "total_processed": frame_count + rejected_count,
                    "accepted": frame_count,
                    "rejected": rejected_count
                },
                "debug_info": {
                    "debug_directory": debug_dir,
                    "rejected_directory": rejected_dir,
                    "visualization_files": [
                        "01_grayscale.png - Original converted to grayscale",
                        "02_blurred.png - Gaussian blur applied", 
                        "03_edges.png - Canny edge detection result",
                        "04_contours_detected.png - ALL CONTOURS WITH QUALITY ANALYSIS!",
                        "05_warped_result.png - Final homography result (if successful)",
                        "success_info.txt - Quality metrics for successful transforms",
                        "failure_reason.txt - Detailed rejection analysis",
                        "contour_info.txt - Complete contour analysis"
                    ]
                }
            }
            yield f"data: {json.dumps(summary_data)}\n\n"
            
            yield "data: [DONE]\n\n"
        
        return StreamingResponse(
            generate_development_stream(),
            media_type="text/event-stream",
            headers={
                "Cache-Control": "no-cache",
                "Connection": "keep-alive",
                "Content-Type": "text/event-stream",
                "Access-Control-Allow-Origin": "*",
                "Access-Control-Allow-Methods": "*",
                "Access-Control-Allow-Headers": "*"
            }
        )
        
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()
        return JSONResponse({
            "success": False,
            "error": f"Development processing failed: {str(e)}"
        })

# Start server
async def start_development_server():
    import uvicorn
    try:
        print("🚀 Starting ENHANCED DEVELOPMENT server on http://localhost:8000")
        print("📁 Images organized: before_homography → after_homography → panorama")
        print("🔧 Document scanner homography with QUALITY CONTROL!")
        print("❌ Frame rejection for poor quality transforms!")
        print("🖼️ Panorama stitching from ACCEPTED frames only!")
        print("🔍 DEBUG VISUALIZATION with quality metrics!")
        print("📂 Directory structure: development/TIMESTAMP/[before|after|rejected|panorama|debug]/")
        print("🎯 Check debug_visualization AND rejected_homography folders!")
        print("💰 NO LLM calls - pure computer vision with quality control!")
        
        config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
        server = uvicorn.Server(config)
        await server.serve()
    except Exception as e:
        print(f"❌ Server error: {e}")

await start_development_server()

🚀 Starting ENHANCED DEVELOPMENT server on http://localhost:8000
📁 Images organized: before_homography → after_homography → panorama
🔧 Document scanner homography with QUALITY CONTROL!
❌ Frame rejection for poor quality transforms!
🖼️ Panorama stitching from ACCEPTED frames only!
🔍 DEBUG VISUALIZATION with quality metrics!
📂 Directory structure: development/TIMESTAMP/[before|after|rejected|panorama|debug]/
🎯 Check debug_visualization AND rejected_homography folders!
💰 NO LLM calls - pure computer vision with quality control!


INFO:     Started server process [28784]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


🔄 DEVELOPMENT FRAME PROCESSING WITH HOMOGRAPHY & PANORAMA & REJECTION...
🔄 DEVELOPMENT FRAME PROCESSING WITH HOMOGRAPHY & PANORAMA & REJECTION...
📥 Form data items: 11
📥 Form data items: 11
📝 Processing frame_0: 1661981 bytes
✅ Processing frame_0
🔍 Found 5 contours for frame_0
  Contour 0: Area=197836, Perimeter=1802, Points=4
  ✅ Found valid 4-point contour at index 0!
    Area: 197836, Aspect ratio: 1.03, Angles: ['87.4°', '90.7°', '91.7°', '90.3°']
✅ Applying homography transformation for frame_0
✅ ACCEPTED frame_0 - homography successful
📝 Processing frame_0: 1661981 bytes
✅ Processing frame_0
🔍 Found 5 contours for frame_0
  Contour 0: Area=197836, Perimeter=1802, Points=4
  ✅ Found valid 4-point contour at index 0!
    Area: 197836, Aspect ratio: 1.03, Angles: ['87.4°', '90.7°', '91.7°', '90.3°']
✅ Applying homography transformation for frame_0
✅ ACCEPTED frame_0 - homography successful
📝 Processing frame_1: 1725969 bytes
✅ Processing frame_1
🔍 Found 5 contours for frame_1
  Cont

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [28784]


with ignore

In [3]:
import nest_asyncio
nest_asyncio.apply()

from fastapi import FastAPI, Request
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse, StreamingResponse
import json
import asyncio
import io
import os
from datetime import datetime
import time
from PIL import Image
import cv2
import numpy as np

app = FastAPI()

# Enable CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Image processor with enhanced homography and panorama functionality
class ImageProcessor:
    @staticmethod
    def order_points(pts):
        """Order points in the order: top-left, top-right, bottom-right, bottom-left"""
        rect = np.zeros((4, 2), dtype="float32")
        
        # Sum and difference to find corners
        s = pts.sum(axis=1)
        diff = np.diff(pts, axis=1)
        
        rect[0] = pts[np.argmin(s)]      # top-left
        rect[2] = pts[np.argmax(s)]      # bottom-right
        rect[1] = pts[np.argmin(diff)]   # top-right
        rect[3] = pts[np.argmax(diff)]   # bottom-left
        
        return rect
    
    @staticmethod
    def four_point_transform(image, pts):
        """Apply perspective transform to get bird's eye view"""
        rect = ImageProcessor.order_points(pts)
        (tl, tr, br, bl) = rect
        
        # Compute width and height of new image
        widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
        widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
        maxWidth = max(int(widthA), int(widthB))
        
        heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
        heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
        maxHeight = max(int(heightA), int(heightB))
        
        # Construct destination points
        dst = np.array([
            [0, 0],
            [maxWidth - 1, 0],
            [maxWidth - 1, maxHeight - 1],
            [0, maxHeight - 1]
        ], dtype="float32")
        
        # Compute perspective transform matrix and apply it
        M = cv2.getPerspectiveTransform(rect, dst)
        warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
        
        return warped
    
    @staticmethod
    def crop_header_footer(image, top_crop=0, bottom_crop=0):
        """Crop top and bottom pixels from image to remove header/footer"""
        try:
            if isinstance(image, Image.Image):
                # PIL Image
                width, height = image.size
                if height > (top_crop + bottom_crop + 100):  # Ensure we don't crop too much
                    cropped = image.crop((0, top_crop, width, height - bottom_crop))
                    print(f"  📏 Cropped header/footer: {width}x{height} → {cropped.width}x{cropped.height}")
                    return cropped
                else:
                    print(f"  ⚠️ Image too small to crop header/footer: {width}x{height}")
                    return image
            else:
                # OpenCV/numpy array
                height, width = image.shape[:2]
                if height > (top_crop + bottom_crop + 100):  # Ensure we don't crop too much
                    cropped = image[top_crop:height-bottom_crop, :]
                    print(f"  📏 Cropped header/footer: {width}x{height} → {cropped.shape[1]}x{cropped.shape[0]}")
                    return cropped
                else:
                    print(f"  ⚠️ Image too small to crop header/footer: {width}x{height}")
                    return image
        except Exception as e:
            print(f"  ❌ Error cropping header/footer: {e}")
            return image
    
    @staticmethod
    def document_scanner_homography(img, save_debug=True, debug_dir=None, frame_key="unknown", crop_header_footer=True):
        """Apply document scanner homography transformation with quality checks and rejection"""
        try:
            # Convert PIL to OpenCV
            img_cv = np.array(img)
            if len(img_cv.shape) == 3:
                img_cv = cv2.cvtColor(img_cv, cv2.COLOR_RGB2BGR)
            
            original = img_cv.copy()
            gray = cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY)
            
            # Apply Gaussian blur
            blurred = cv2.GaussianBlur(gray, (5, 5), 0)
            
            # Edge detection
            edged = cv2.Canny(blurred, 75, 200)
            
            # Save debug images if requested
            if save_debug and debug_dir:
                debug_frame_dir = os.path.join(debug_dir, frame_key)
                os.makedirs(debug_frame_dir, exist_ok=True)
                
                # Save processing steps
                cv2.imwrite(os.path.join(debug_frame_dir, "01_grayscale.png"), gray)
                cv2.imwrite(os.path.join(debug_frame_dir, "02_blurred.png"), blurred)
                cv2.imwrite(os.path.join(debug_frame_dir, "03_edges.png"), edged)
            
            # Find contours
            contours, _ = cv2.findContours(edged, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
            contours = sorted(contours, key=cv2.contourArea, reverse=True)[:5]
            
            print(f"🔍 Found {len(contours)} contours for {frame_key}")
            
            # Create debug image with all contours
            debug_contours = original.copy()
            cv2.drawContours(debug_contours, contours, -1, (0, 255, 0), 2)
            
            # Find the largest contour with 4 points (document)
            screenCnt = None
            contour_info = []
            rejection_reason = None
            
            for idx, c in enumerate(contours):
                area = cv2.contourArea(c)
                peri = cv2.arcLength(c, True)
                approx = cv2.approxPolyDP(c, 0.02 * peri, True)
                
                contour_info.append({
                    "index": idx,
                    "area": area,
                    "perimeter": peri,
                    "points": len(approx)
                })
                
                print(f"  Contour {idx}: Area={area:.0f}, Perimeter={peri:.0f}, Points={len(approx)}")
                
                # Draw contour number on debug image
                if len(approx) >= 3:
                    M = cv2.moments(c)
                    if M["m00"] != 0:
                        cx = int(M["m10"] / M["m00"])
                        cy = int(M["m01"] / M["m00"])
                        cv2.putText(debug_contours, f"{idx}({len(approx)}p)", (cx-20, cy), 
                                  cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
                
                if len(approx) == 4:
                    # QUALITY CHECKS FOR 4-POINT CONTOUR
                    
                    # 1. Area check - contour should be reasonably large
                    min_area = (img_cv.shape[0] * img_cv.shape[1]) * 0.1  # At least 10% of image
                    if area < min_area:
                        rejection_reason = f"contour_too_small_area_{area:.0f}_min_{min_area:.0f}"
                        print(f"  ❌ Contour {idx} rejected: too small (area: {area:.0f} < {min_area:.0f})")
                        continue
                    
                    # 2. Aspect ratio check - should look reasonable
                    rect = ImageProcessor.order_points(approx.reshape(4, 2))
                    (tl, tr, br, bl) = rect
                    
                    width1 = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
                    width2 = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
                    height1 = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
                    height2 = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
                    
                    avg_width = (width1 + width2) / 2
                    avg_height = (height1 + height2) / 2
                    aspect_ratio = max(avg_width, avg_height) / min(avg_width, avg_height)
                    
                    if aspect_ratio > 5.0:  # Too extreme aspect ratio
                        rejection_reason = f"extreme_aspect_ratio_{aspect_ratio:.2f}"
                        print(f"  ❌ Contour {idx} rejected: extreme aspect ratio ({aspect_ratio:.2f})")
                        continue
                    
                    # 3. Corner angle check - corners should be roughly 90 degrees
                    def angle_between_vectors(v1, v2):
                        cos_angle = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
                        cos_angle = np.clip(cos_angle, -1.0, 1.0)
                        return np.degrees(np.arccos(cos_angle))
                    
                    corners = approx.reshape(4, 2)
                    angles = []
                    for i in range(4):
                        p1 = corners[i]
                        p2 = corners[(i + 1) % 4]
                        p3 = corners[(i + 2) % 4]
                        
                        v1 = p1 - p2
                        v2 = p3 - p2
                        angle = angle_between_vectors(v1, v2)
                        angles.append(angle)
                    
                    # Check if any angle is too far from 90 degrees
                    angle_threshold = 45  # degrees deviation from 90
                    bad_angles = [abs(angle - 90) for angle in angles if abs(angle - 90) > angle_threshold]
                    
                    if bad_angles:
                        rejection_reason = f"bad_corner_angles_{bad_angles}"
                        print(f"  ❌ Contour {idx} rejected: bad corner angles {angles}")
                        continue
                    
                    print(f"  ✅ Found valid 4-point contour at index {idx}!")
                    print(f"    Area: {area:.0f}, Aspect ratio: {aspect_ratio:.2f}, Angles: {[f'{a:.1f}°' for a in angles]}")
                    
                    screenCnt = approx
                    
                    # Draw the selected contour in red
                    cv2.drawContours(debug_contours, [approx], -1, (0, 0, 255), 3)
                    
                    # Draw corner points
                    for i, point in enumerate(approx):
                        cv2.circle(debug_contours, tuple(point[0]), 8, (255, 255, 0), -1)
                        cv2.putText(debug_contours, str(i), tuple(point[0] + 10), 
                                  cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
                    break
            
            # Save debug images
            if save_debug and debug_dir:
                cv2.imwrite(os.path.join(debug_frame_dir, "04_contours_detected.png"), debug_contours)
                
                # Save contour info as text
                with open(os.path.join(debug_frame_dir, "contour_info.txt"), "w") as f:
                    f.write(f"Contour Analysis for {frame_key}\n")
                    f.write("=" * 40 + "\n")
                    for info in contour_info:
                        f.write(f"Contour {info['index']}: Area={info['area']:.0f}, "
                               f"Perimeter={info['perimeter']:.0f}, Points={info['points']}\n")
                    f.write(f"\nSelected contour: {'Found' if screenCnt is not None else 'None'}\n")
                    if rejection_reason:
                        f.write(f"Rejection reason: {rejection_reason}\n")
            
            if screenCnt is not None:
                print(f"✅ Applying homography transformation for {frame_key}")
                
                # Apply the four point transform
                warped = ImageProcessor.four_point_transform(original, screenCnt.reshape(4, 2))
                
                # POST-WARP QUALITY CHECKS
                
                # 1. Check if warped image is too small
                min_warped_size = 200  # pixels
                if warped.shape[0] < min_warped_size or warped.shape[1] < min_warped_size:
                    rejection_reason = f"warped_too_small_{warped.shape[1]}x{warped.shape[0]}"
                    print(f"  ❌ Warped result rejected: too small ({warped.shape[1]}x{warped.shape[0]})")
                    raise ValueError(f"Warped image too small: {warped.shape}")
                
                # 2. Check if warped image is extremely distorted
                warp_aspect = max(warped.shape[0], warped.shape[1]) / min(warped.shape[0], warped.shape[1])
                if warp_aspect > 10.0:
                    rejection_reason = f"warped_extreme_aspect_{warp_aspect:.2f}"
                    print(f"  ❌ Warped result rejected: extreme aspect ratio ({warp_aspect:.2f})")
                    raise ValueError(f"Warped image too distorted: aspect ratio {warp_aspect:.2f}")
                
                # 3. Check if warped image is mostly black/empty
                gray_warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
                non_zero_pixels = np.count_nonzero(gray_warped > 50)  # Pixels brighter than 50
                total_pixels = gray_warped.shape[0] * gray_warped.shape[1]
                content_ratio = non_zero_pixels / total_pixels
                
                if content_ratio < 0.3:  # Less than 30% content
                    rejection_reason = f"warped_mostly_empty_{content_ratio:.2f}"
                    print(f"  ❌ Warped result rejected: mostly empty ({content_ratio:.2%} content)")
                    raise ValueError(f"Warped image mostly empty: {content_ratio:.2%} content")
                
                # Save warped result BEFORE cropping
                if save_debug and debug_dir:
                    cv2.imwrite(os.path.join(debug_frame_dir, "05_warped_result.png"), warped)
                
                # CROP HEADER AND FOOTER AFTER WARPING
                warped_cropped = warped
                if crop_header_footer:
                    print(f"  ✂️ Cropping header/footer from warped image for {frame_key}")
                    warped_cropped = ImageProcessor.crop_header_footer(warped, top_crop=0, bottom_crop=0)
                    
                    # Save cropped version for debugging
                    if save_debug and debug_dir:
                        cv2.imwrite(os.path.join(debug_frame_dir, "06_warped_cropped.png"), warped_cropped)
                
                # Update quality checks for cropped image
                if crop_header_footer:
                    # Re-check if cropped image is too small
                    if warped_cropped.shape[0] < min_warped_size or warped_cropped.shape[1] < min_warped_size:
                        rejection_reason = f"cropped_too_small_{warped_cropped.shape[1]}x{warped_cropped.shape[0]}"
                        print(f"  ❌ Cropped result rejected: too small ({warped_cropped.shape[1]}x{warped_cropped.shape[0]})")
                        raise ValueError(f"Cropped image too small: {warped_cropped.shape}")
                
                # Save success info
                if save_debug and debug_dir:
                    with open(os.path.join(debug_frame_dir, "success_info.txt"), "w") as f:
                        f.write(f"Homography successful for {frame_key}\n")
                        f.write(f"Original warped size: {warped.shape[1]}x{warped.shape[0]}\n")
                        f.write(f"Cropped size: {warped_cropped.shape[1]}x{warped_cropped.shape[0]}\n")
                        f.write(f"Warped aspect ratio: {warp_aspect:.2f}\n")
                        f.write(f"Content ratio: {content_ratio:.2%}\n")
                        f.write(f"Header/footer cropped: {'Yes' if crop_header_footer else 'No'}\n")
                
                # Convert back to RGB for PIL (use cropped version)
                if len(warped_cropped.shape) == 3:
                    warped_cropped = cv2.cvtColor(warped_cropped, cv2.COLOR_BGR2RGB)
                
                return Image.fromarray(warped_cropped), "success"
            else:
                if not rejection_reason:
                    rejection_reason = "no_4_point_contour_found"
                
                print(f"⚠️ No valid 4-point contour found for {frame_key}")
                
                # Save info about why it failed
                if save_debug and debug_dir:
                    with open(os.path.join(debug_frame_dir, "failure_reason.txt"), "w") as f:
                        f.write(f"Homography failed for {frame_key}\n")
                        f.write(f"Reason: {rejection_reason}\n\n")
                        f.write("Suggestions:\n")
                        f.write("1. Check if image has clear document edges\n")
                        f.write("2. Adjust Canny edge detection parameters\n")
                        f.write("3. Try different contour approximation epsilon\n")
                        f.write("4. Ensure good contrast between document and background\n")
                        f.write("5. Make sure document occupies significant portion of image\n")
                
                return None, rejection_reason
                
        except Exception as e:
            error_msg = f"error_{str(e).replace(' ', '_')}"
            print(f"❌ Homography error for {frame_key}: {e}")
            
            if save_debug and debug_dir:
                debug_frame_dir = os.path.join(debug_dir, frame_key)
                os.makedirs(debug_frame_dir, exist_ok=True)
                with open(os.path.join(debug_frame_dir, "error.txt"), "w") as f:
                    f.write(f"Error processing {frame_key}: {str(e)}\n")
            
            return None, error_msg
    

@app.post("/process-multiple-frames-stream")
async def process_frames_development(request: Request):
    """Development endpoint with homography and panorama functionality with frame rejection and header/footer cropping"""
    
    try:
        print("=" * 50)
        print("🔄 DEVELOPMENT FRAME PROCESSING WITH HOMOGRAPHY & PANORAMA & REJECTION & HEADER/FOOTER CROPPING...")
        
        # Create directory structure
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        base_dir = "development"
        session_dir = os.path.join(base_dir, timestamp)
        before_dir = os.path.join(session_dir, "before_homography")
        after_dir = os.path.join(session_dir, "after_homography")
        rejected_dir = os.path.join(session_dir, "rejected_homography")
        debug_dir = os.path.join(session_dir, "debug_visualization")
        
        os.makedirs(before_dir, exist_ok=True)
        os.makedirs(after_dir, exist_ok=True)
        os.makedirs(rejected_dir, exist_ok=True)
        os.makedirs(debug_dir, exist_ok=True)
        
        # Parse form data
        form_data = await request.form()
        print(f"📥 Form data items: {len(form_data)}")
        
        # Lists to store all images
        all_images = []
        rejected_images = []
        processing_results = []
        frame_count = 0
        rejected_count = 0
        
        for key, value in form_data.items():
            if key.startswith('frame_') and hasattr(value, 'read'):
                try:
                    # Read file data
                    file_data = await value.read()
                    print(f"📝 Processing {key}: {len(file_data)} bytes")
                    
                    # Open original image directly from file data
                    original_img = Image.open(io.BytesIO(file_data))
                    
                    print(f"✅ Processing {key}")
                    
                    # Apply document scanner homography with quality checks AND header/footer cropping
                    processed_img, status = ImageProcessor.document_scanner_homography(
                        original_img, 
                        save_debug=True, 
                        debug_dir=debug_dir, 
                        frame_key=key,
                        crop_header_footer=True  # Enable header/footer cropping
                    )
                    
                    if processed_img is not None and status == "success":
                        # SUCCESS - Add to accepted processing results
                        image_data = {
                            "key": key,
                            "original_image": original_img,
                            "processed_image": processed_img,  # This is now cropped
                            "file_data": file_data,
                            "filename": getattr(value, 'filename', 'unknown'),
                            "size_bytes": len(file_data),
                            "image_size": f"{original_img.width}x{original_img.height}",
                            "processed_size": f"{processed_img.width}x{processed_img.height}",
                            "status": "accepted"
                        }
                        all_images.append(image_data)
                        
                        # Save both images
                        before_path = os.path.join(before_dir, f"{key}_original.png")
                        after_path = os.path.join(after_dir, f"{key}_homography_cropped.png")
                        
                        original_img.save(before_path)
                        processed_img.save(after_path)
                        
                        processing_results.append({
                            "frame_key": key,
                            "original_path": before_path,
                            "processed_path": after_path,
                            "debug_path": os.path.join(debug_dir, key),
                            "original_size": f"{original_img.width}x{original_img.height}",
                            "processed_size": f"{processed_img.width}x{processed_img.height}",
                            "original_file_size": len(file_data),
                            "processed_file_size": os.path.getsize(after_path),
                            "status": "accepted",
                            "cropped": True
                        })
                        
                        frame_count += 1
                        print(f"✅ ACCEPTED {key} - homography + cropping successful")
                    else:
                        # REJECTED - Save to rejected folder
                        rejected_path = os.path.join(rejected_dir, f"{key}_rejected_{status}.png")
                        original_img.save(rejected_path)
                        
                        rejected_images.append({
                            "key": key,
                            "filename": getattr(value, 'filename', 'unknown'),
                            "size_bytes": len(file_data),
                            "image_size": f"{original_img.width}x{original_img.height}",
                            "rejection_reason": status,
                            "rejected_path": rejected_path,
                            "debug_path": os.path.join(debug_dir, key)
                        })
                        
                        rejected_count += 1
                        print(f"❌ REJECTED {key} - {status}")
                        
                except Exception as frame_error:
                    print(f"❌ Error processing {key}: {frame_error}")
        
        
        print(f"🎉 Successfully processed {frame_count} frames!")
        print(f"❌ Rejected {rejected_count} frames!")
        print(f"📋 {len(all_images)} accepted images loaded into list for testing")
        
        # Create streaming response
        async def generate_development_stream():
            # Initial response
            initial_response = {
                "success": True,
                "message": f"Development processing complete! {frame_count} accepted, {rejected_count} rejected (with header/footer cropping)",
                "timestamp": timestamp,
                "session_directory": session_dir,
                "directories": {
                    "session": session_dir,
                    "before_homography": before_dir,
                    "after_homography": after_dir,
                    "rejected_homography": rejected_dir,
                    "debug_visualization": debug_dir
                },
                "frame_count": frame_count,
                "rejected_count": rejected_count,
                "all_images_count": len(all_images),
                "header_footer_cropped": True,
                "type": "initial"
            }
            yield f"data: {json.dumps(initial_response)}\n\n"
            
            await asyncio.sleep(0.3)
            
            # Stream processing results for accepted frames
            for i, result in enumerate(processing_results):
                stream_data = {
                    "type": "frame_result",
                    "frame_index": i + 1,
                    "total_frames": frame_count,
                    "result": result
                }
                yield f"data: {json.dumps(stream_data)}\n\n"
                await asyncio.sleep(0.1)
            
            # Stream rejected frames info
            for i, rejected in enumerate(rejected_images):
                stream_data = {
                    "type": "rejected_frame",
                    "rejected_index": i + 1,
                    "total_rejected": rejected_count,
                    "result": rejected
                }
                yield f"data: {json.dumps(stream_data)}\n\n"
                await asyncio.sleep(0.1)
            
            # Summary
            total_original_size = sum([img["size_bytes"] for img in all_images])
            total_rejected_size = sum([img["size_bytes"] for img in rejected_images])
            
            summary_data = {
                "type": "summary",
                "content": f"""🔬 **Advanced Development Processing with Frame Rejection & Header/Footer Cropping**

📊 **Frame Statistics:**
- ✅ Total frames accepted: {frame_count}
- ❌ Total frames rejected: {rejected_count}
- 📈 Acceptance rate: {(frame_count/(frame_count+rejected_count)*100):.1f}%
- ✂️ Header/footer cropping: ✅ Enabled (50px top + bottom)

📁 **Data Size:**
- Accepted frames: {total_original_size:,} bytes
- Rejected frames: {total_rejected_size:,} bytes
- Total processed: {(total_original_size + total_rejected_size):,} bytes

📍 **Session Directory Structure:**
   - Main session: {session_dir}
   - ✅ Before homography: {before_dir}
   - ✅ After homography: {after_dir} **(cropped)**
   - ❌ **Rejected frames: {rejected_dir}**
   - 🔍 **Debug visualization: {debug_dir}**

🖼️ **Images in Memory:**
- Accepted images: {len(all_images)} (all cropped)
- Each image contains: PIL Image objects, metadata
- Ready for testing and development

🔧 **Enhanced Processing Flow:**
1. Read file data from form ✅
2. Create PIL Image directly from bytes ✅
3. Apply document scanner homography with quality checks ✅
4. **🔍 Quality validation (area, aspect ratio, corner angles) ✅**
5. **📊 Post-warp validation (size, distortion, content) ✅**
6. **✂️ Crop header/footer (50px top + bottom) ✅**
7. **❌ Reject poor quality transforms ✅**
8. **🔍 Save debug visualization images ✅**
9. Create panorama from ACCEPTED + CROPPED images only ✅
10. Add to organized directory structure ✅

📂 **Directory Organization:**
development/
└── {timestamp}/
    ├── before_homography/ (original only)
    ├── after_homography/ (accepted + cropped)
    ├── **rejected_homography/** ❌
    ├── panorama/ (stitched from cropped images)
    └── **debug_visualization/**
        └── **frame_X/**
            ├── **01_grayscale.png**
            ├── **02_blurred.png**
            ├── **03_edges.png**
            ├── **04_contours_detected.png** (with analysis!)
            ├── **05_warped_result.png** (before crop)
            ├── **06_warped_cropped.png** ✂️ (after crop)
            ├── **success_info.txt** (with crop info)
            ├── **failure_reason.txt** (if failed)
            └── **contour_info.txt**

🔍 **Quality Checks Implemented:**
- ✅ **Pre-warp validation:**
  - Minimum contour area (10% of image)
  - Reasonable aspect ratio (< 5:1)
  - Corner angles close to 90° (±45°)
- ✅ **Post-warp validation:**
  - Minimum warped size (200px)
  - Maximum aspect ratio (< 10:1)
  - Content density (> 30% non-black pixels)
- ✅ **Post-crop validation:**
  - Minimum cropped size (200px after removing header/footer)

✂️ **Header/Footer Cropping:**
- **Top crop: 50px** (removes headers, navigation bars)
- **Bottom crop: 50px** (removes footers, buttons)
- Applied AFTER homography transformation
- Helps panorama stitching by removing UI elements
- Both uncropped and cropped versions saved for debugging

❌ **Rejection Reasons Tracked:**
- contour_too_small_area_X_min_Y
- extreme_aspect_ratio_X
- bad_corner_angles_[list]
- warped_too_small_WxH
- warped_extreme_aspect_X
- warped_mostly_empty_X%
- **cropped_too_small_WxH** ✂️
- no_4_point_contour_found
- error_[exception_details]

💡 **Debug Features:**
- **Check {debug_dir} for detailed analysis!**
- Each frame has complete processing visualization
- **06_warped_cropped.png shows final result for panorama**
- Success/failure reasons clearly documented
- Quality metrics saved for analysis
- Rejected frames preserved for inspection

🚀 **Production-ready homography pipeline with quality control + header/footer removal for better panorama stitching!**""",
                "total_original_size": total_original_size,
                "total_rejected_size": total_rejected_size,
                "accepted_images_in_memory": len(all_images),
                "rejected_images_count": len(rejected_images),
                "processing_results": processing_results,
                "rejected_results": rejected_images,
                "quality_metrics": {
                    "acceptance_rate": f"{(frame_count/(frame_count+rejected_count)*100):.1f}%" if (frame_count+rejected_count) > 0 else "N/A",
                    "total_processed": frame_count + rejected_count,
                    "accepted": frame_count,
                    "rejected": rejected_count
                },
                "cropping_info": {
                    "enabled": True,
                    "top_crop_px": 00,
                    "bottom_crop_px": 00,
                    "purpose": "Remove headers and footers for better panorama stitching"
                },
                "debug_info": {
                    "debug_directory": debug_dir,
                    "rejected_directory": rejected_dir,
                    "visualization_files": [
                        "01_grayscale.png - Original converted to grayscale",
                        "02_blurred.png - Gaussian blur applied", 
                        "03_edges.png - Canny edge detection result",
                        "04_contours_detected.png - ALL CONTOURS WITH QUALITY ANALYSIS!",
                        "05_warped_result.png - Homography result (before crop)",
                        "06_warped_cropped.png - Final result after header/footer crop ✂️",
                        "success_info.txt - Quality metrics including crop info",
                        "failure_reason.txt - Detailed rejection analysis",
                        "contour_info.txt - Complete contour analysis"
                    ]
                }
            }
            yield f"data: {json.dumps(summary_data)}\n\n"
            
            yield "data: [DONE]\n\n"
        
        return StreamingResponse(
            generate_development_stream(),
            media_type="text/event-stream",
            headers={
                "Cache-Control": "no-cache",
                "Connection": "keep-alive",
                "Content-Type": "text/event-stream",
                "Access-Control-Allow-Origin": "*",
                "Access-Control-Allow-Methods": "*",
                "Access-Control-Allow-Headers": "*"
            }
        )
        
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()
        return JSONResponse({
            "success": False,
            "error": f"Development processing failed: {str(e)}"
        })

# Start server
async def start_development_server():
    import uvicorn
    try:
        print("🚀 Starting ENHANCED DEVELOPMENT server on http://localhost:8000")
        print("📁 Images organized: before_homography → after_homography → panorama")
        print("🔧 Document scanner homography with QUALITY CONTROL!")
        print("❌ Frame rejection for poor quality transforms!")
        print("🔍 DEBUG VISUALIZATION with quality metrics!")
        print("📂 Directory structure: development/TIMESTAMP/[before|after|rejected|debug]/")
        
        config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
        server = uvicorn.Server(config)
        await server.serve()
    except Exception as e:
        print(f"❌ Server error: {e}")

await start_development_server()

🚀 Starting ENHANCED DEVELOPMENT server on http://localhost:8000
📁 Images organized: before_homography → after_homography → panorama
🔧 Document scanner homography with QUALITY CONTROL!
❌ Frame rejection for poor quality transforms!
🔍 DEBUG VISUALIZATION with quality metrics!
📂 Directory structure: development/TIMESTAMP/[before|after|rejected|debug]/


INFO:     Started server process [28020]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [28020]


FINALLL

In [6]:
import nest_asyncio
nest_asyncio.apply()

from fastapi import FastAPI, Request
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse, StreamingResponse
import json
import asyncio
import io
import os
from datetime import datetime
import time
from PIL import Image
import cv2
import numpy as np

app = FastAPI()

# Enable CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# ...existing code...

class ImageProcessor:
    @staticmethod
    def order_points(pts):
        """Order points in the order: top-left, top-right, bottom-right, bottom-left"""
        rect = np.zeros((4, 2), dtype="float32")
        s = pts.sum(axis=1)
        diff = np.diff(pts, axis=1)
        rect[0] = pts[np.argmin(s)]      # top-left
        rect[2] = pts[np.argmax(s)]      # bottom-right
        rect[1] = pts[np.argmin(diff)]   # top-right
        rect[3] = pts[np.argmax(diff)]   # bottom-left
        return rect

    @staticmethod
    def four_point_transform(image, pts):
        """Apply perspective transform to get bird's eye view"""
        rect = ImageProcessor.order_points(pts)
        (tl, tr, br, bl) = rect
        widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
        widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
        maxWidth = max(int(widthA), int(widthB))
        heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
        heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
        maxHeight = max(int(heightA), int(heightB))
        dst = np.array([
            [0, 0],
            [maxWidth - 1, 0],
            [maxWidth - 1, maxHeight - 1],
            [0, maxHeight - 1]
        ], dtype="float32")
        M = cv2.getPerspectiveTransform(rect, dst)
        warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
        return warped

    @staticmethod
    def document_scanner_homography(img, save_debug=True, debug_dir=None, frame_key="unknown"):
        """Apply document scanner homography transformation with quality checks and rejection"""
        try:
            img_cv = np.array(img)
            if len(img_cv.shape) == 3:
                img_cv = cv2.cvtColor(img_cv, cv2.COLOR_RGB2BGR)
            original = img_cv.copy()
            gray = cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY)
            blurred = cv2.GaussianBlur(gray, (5, 5), 0)
            edged = cv2.Canny(blurred, 75, 200)
            if save_debug and debug_dir:
                debug_frame_dir = os.path.join(debug_dir, frame_key)
                os.makedirs(debug_frame_dir, exist_ok=True)
                cv2.imwrite(os.path.join(debug_frame_dir, "01_grayscale.png"), gray)
                cv2.imwrite(os.path.join(debug_frame_dir, "02_blurred.png"), blurred)
                cv2.imwrite(os.path.join(debug_frame_dir, "03_edges.png"), edged)
            contours, _ = cv2.findContours(edged, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
            contours = sorted(contours, key=cv2.contourArea, reverse=True)[:5]
            print(f"🔍 Found {len(contours)} contours for {frame_key}")
            debug_contours = original.copy()
            cv2.drawContours(debug_contours, contours, -1, (0, 255, 0), 2)
            screenCnt = None
            contour_info = []
            rejection_reason = None
            for idx, c in enumerate(contours):
                area = cv2.contourArea(c)
                peri = cv2.arcLength(c, True)
                approx = cv2.approxPolyDP(c, 0.02 * peri, True)
                contour_info.append({
                    "index": idx,
                    "area": area,
                    "perimeter": peri,
                    "points": len(approx)
                })
                print(f"  Contour {idx}: Area={area:.0f}, Perimeter={peri:.0f}, Points={len(approx)}")
                if len(approx) >= 3:
                    M = cv2.moments(c)
                    if M["m00"] != 0:
                        cx = int(M["m10"] / M["m00"])
                        cy = int(M["m01"] / M["m00"])
                        cv2.putText(debug_contours, f"{idx}({len(approx)}p)", (cx-20, cy),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
                if len(approx) == 4:
                    min_area = (img_cv.shape[0] * img_cv.shape[1]) * 0.1
                    if area < min_area:
                        rejection_reason = f"contour_too_small_area_{area:.0f}_min_{min_area:.0f}"
                        print(f"  ❌ Contour {idx} rejected: too small (area: {area:.0f} < {min_area:.0f})")
                        continue
                    rect = ImageProcessor.order_points(approx.reshape(4, 2))
                    (tl, tr, br, bl) = rect
                    width1 = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
                    width2 = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
                    height1 = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
                    height2 = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
                    avg_width = (width1 + width2) / 2
                    avg_height = (height1 + height2) / 2
                    aspect_ratio = max(avg_width, avg_height) / min(avg_width, avg_height)
                    if aspect_ratio > 5.0:
                        rejection_reason = f"extreme_aspect_ratio_{aspect_ratio:.2f}"
                        print(f"  ❌ Contour {idx} rejected: extreme aspect ratio ({aspect_ratio:.2f})")
                        continue
                    def angle_between_vectors(v1, v2):
                        cos_angle = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
                        cos_angle = np.clip(cos_angle, -1.0, 1.0)
                        return np.degrees(np.arccos(cos_angle))
                    corners = approx.reshape(4, 2)
                    angles = []
                    for i in range(4):
                        p1 = corners[i]
                        p2 = corners[(i + 1) % 4]
                        p3 = corners[(i + 2) % 4]
                        v1 = p1 - p2
                        v2 = p3 - p2
                        angle = angle_between_vectors(v1, v2)
                        angles.append(angle)
                    angle_threshold = 45
                    bad_angles = [abs(angle - 90) for angle in angles if abs(angle - 90) > angle_threshold]
                    if bad_angles:
                        rejection_reason = f"bad_corner_angles_{bad_angles}"
                        print(f"  ❌ Contour {idx} rejected: bad corner angles {angles}")
                        continue
                    print(f"  ✅ Found valid 4-point contour at index {idx}!")
                    print(f"    Area: {area:.0f}, Aspect ratio: {aspect_ratio:.2f}, Angles: {[f'{a:.1f}°' for a in angles]}")
                    screenCnt = approx
                    cv2.drawContours(debug_contours, [approx], -1, (0, 0, 255), 3)
                    for i, point in enumerate(approx):
                        cv2.circle(debug_contours, tuple(point[0]), 8, (255, 255, 0), -1)
                        cv2.putText(debug_contours, str(i), tuple(point[0] + 10),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
                    break
            if save_debug and debug_dir:
                cv2.imwrite(os.path.join(debug_frame_dir, "04_contours_detected.png"), debug_contours)
                with open(os.path.join(debug_frame_dir, "contour_info.txt"), "w") as f:
                    f.write(f"Contour Analysis for {frame_key}\n")
                    f.write("=" * 40 + "\n")
                    for info in contour_info:
                        f.write(f"Contour {info['index']}: Area={info['area']:.0f}, "
                                f"Perimeter={info['perimeter']:.0f}, Points={info['points']}\n")
                    f.write(f"\nSelected contour: {'Found' if screenCnt is not None else 'None'}\n")
                    if rejection_reason:
                        f.write(f"Rejection reason: {rejection_reason}\n")
            if screenCnt is not None:
                print(f"✅ Applying homography transformation for {frame_key}")
                warped = ImageProcessor.four_point_transform(original, screenCnt.reshape(4, 2))
                min_warped_size = 200
                if warped.shape[0] < min_warped_size or warped.shape[1] < min_warped_size:
                    rejection_reason = f"warped_too_small_{warped.shape[1]}x{warped.shape[0]}"
                    print(f"  ❌ Warped result rejected: too small ({warped.shape[1]}x{warped.shape[0]})")
                    raise ValueError(f"Warped image too small: {warped.shape}")
                warp_aspect = max(warped.shape[0], warped.shape[1]) / min(warped.shape[0], warped.shape[1])
                if warp_aspect > 10.0:
                    rejection_reason = f"warped_extreme_aspect_{warp_aspect:.2f}"
                    print(f"  ❌ Warped result rejected: extreme aspect ratio ({warp_aspect:.2f})")
                    raise ValueError(f"Warped image too distorted: aspect ratio {warp_aspect:.2f}")
                gray_warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
                non_zero_pixels = np.count_nonzero(gray_warped > 50)
                total_pixels = gray_warped.shape[0] * gray_warped.shape[1]
                content_ratio = non_zero_pixels / total_pixels
                if content_ratio < 0.3:
                    rejection_reason = f"warped_mostly_empty_{content_ratio:.2f}"
                    print(f"  ❌ Warped result rejected: mostly empty ({content_ratio:.2%} content)")
                    raise ValueError(f"Warped image mostly empty: {content_ratio:.2%} content")
                if save_debug and debug_dir:
                    cv2.imwrite(os.path.join(debug_frame_dir, "05_warped_result.png"), warped)
                    with open(os.path.join(debug_frame_dir, "success_info.txt"), "w") as f:
                        f.write(f"Homography successful for {frame_key}\n")
                        f.write(f"Warped size: {warped.shape[1]}x{warped.shape[0]}\n")
                        f.write(f"Warped aspect ratio: {warp_aspect:.2f}\n")
                        f.write(f"Content ratio: {content_ratio:.2%}\n")
                if len(warped.shape) == 3:
                    warped = cv2.cvtColor(warped, cv2.COLOR_BGR2RGB)
                return Image.fromarray(warped), "success"
            else:
                if not rejection_reason:
                    rejection_reason = "no_4_point_contour_found"
                print(f"⚠️ No valid 4-point contour found for {frame_key}")
                if save_debug and debug_dir:
                    with open(os.path.join(debug_frame_dir, "failure_reason.txt"), "w") as f:
                        f.write(f"Homography failed for {frame_key}\n")
                        f.write(f"Reason: {rejection_reason}\n\n")
                        f.write("Suggestions:\n")
                        f.write("1. Check if image has clear document edges\n")
                        f.write("2. Adjust Canny edge detection parameters\n")
                        f.write("3. Try different contour approximation epsilon\n")
                        f.write("4. Ensure good contrast between document and background\n")
                        f.write("5. Make sure document occupies significant portion of image\n")
                return None, rejection_reason
        except Exception as e:
            error_msg = f"error_{str(e).replace(' ', '_')}"
            print(f"❌ Homography error for {frame_key}: {e}")
            if save_debug and debug_dir:
                debug_frame_dir = os.path.join(debug_dir, frame_key)
                os.makedirs(debug_frame_dir, exist_ok=True)
                with open(os.path.join(debug_frame_dir, "error.txt"), "w") as f:
                    f.write(f"Error processing {frame_key}: {str(e)}\n")
            return None, error_msg

@app.post("/process-multiple-frames-stream")
async def process_frames_development(request: Request):
    """Development endpoint with homography and panorama functionality with frame rejection (NO cropping)"""
    try:
        print("=" * 50)
        print("🔄 DEVELOPMENT FRAME PROCESSING WITH HOMOGRAPHY & PANORAMA & REJECTION (NO CROPPING)...")
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        base_dir = "development"
        session_dir = os.path.join(base_dir, timestamp)
        before_dir = os.path.join(session_dir, "before_homography")
        after_dir = os.path.join(session_dir, "after_homography")
        rejected_dir = os.path.join(session_dir, "rejected_homography")
        debug_dir = os.path.join(session_dir, "debug_visualization")
        os.makedirs(before_dir, exist_ok=True)
        os.makedirs(after_dir, exist_ok=True)
        os.makedirs(rejected_dir, exist_ok=True)
        os.makedirs(debug_dir, exist_ok=True)
        form_data = await request.form()
        print(f"📥 Form data items: {len(form_data)}")
        all_images = []
        rejected_images = []
        processing_results = []
        frame_count = 0
        rejected_count = 0
        for key, value in form_data.items():
            if key.startswith('frame_') and hasattr(value, 'read'):
                try:
                    file_data = await value.read()
                    print(f"📝 Processing {key}: {len(file_data)} bytes")
                    original_img = Image.open(io.BytesIO(file_data))
                    print(f"✅ Processing {key}")
                    processed_img, status = ImageProcessor.document_scanner_homography(
                        original_img,
                        save_debug=True,
                        debug_dir=debug_dir,
                        frame_key=key
                    )
                    if processed_img is not None and status == "success":
                        image_data = {
                            "key": key,
                            "original_image": original_img,
                            "processed_image": processed_img,
                            "file_data": file_data,
                            "filename": getattr(value, 'filename', 'unknown'),
                            "size_bytes": len(file_data),
                            "image_size": f"{original_img.width}x{original_img.height}",
                            "processed_size": f"{processed_img.width}x{processed_img.height}",
                            "status": "accepted"
                        }
                        all_images.append(image_data)
                        before_path = os.path.join(before_dir, f"{key}_original.png")
                        after_path = os.path.join(after_dir, f"{key}_homography.png")
                        original_img.save(before_path)
                        processed_img.save(after_path)
                        processing_results.append({
                            "frame_key": key,
                            "original_path": before_path,
                            "processed_path": after_path,
                            "debug_path": os.path.join(debug_dir, key),
                            "original_size": f"{original_img.width}x{original_img.height}",
                            "processed_size": f"{processed_img.width}x{processed_img.height}",
                            "original_file_size": len(file_data),
                            "processed_file_size": os.path.getsize(after_path),
                            "status": "accepted"
                        })
                        frame_count += 1
                        print(f"✅ ACCEPTED {key} - homography successful")
                    else:
                        rejected_path = os.path.join(rejected_dir, f"{key}_rejected_{status}.png")
                        original_img.save(rejected_path)
                        rejected_images.append({
                            "key": key,
                            "filename": getattr(value, 'filename', 'unknown'),
                            "size_bytes": len(file_data),
                            "image_size": f"{original_img.width}x{original_img.height}",
                            "rejection_reason": status,
                            "rejected_path": rejected_path,
                            "debug_path": os.path.join(debug_dir, key)
                        })
                        rejected_count += 1
                        print(f"❌ REJECTED {key} - {status}")
                except Exception as frame_error:
                    print(f"❌ Error processing {key}: {frame_error}")
        print(f"🎉 Successfully processed {frame_count} frames!")
        print(f"❌ Rejected {rejected_count} frames!")
        print(f"📋 {len(all_images)} accepted images loaded into list for testing")
        async def generate_development_stream():
            initial_response = {
                "success": True,
                "message": f"Development processing complete! {frame_count} accepted, {rejected_count} rejected (NO cropping)",
                "timestamp": timestamp,
                "session_directory": session_dir,
                "directories": {
                    "session": session_dir,
                    "before_homography": before_dir,
                    "after_homography": after_dir,
                    "rejected_homography": rejected_dir,
                    "debug_visualization": debug_dir
                },
                "frame_count": frame_count,
                "rejected_count": rejected_count,
                "all_images_count": len(all_images),
                "header_footer_cropped": False,
                "type": "initial"
            }
            yield f"data: {json.dumps(initial_response)}\n\n"
            await asyncio.sleep(0.3)
            for i, result in enumerate(processing_results):
                stream_data = {
                    "type": "frame_result",
                    "frame_index": i + 1,
                    "total_frames": frame_count,
                    "result": result
                }
                yield f"data: {json.dumps(stream_data)}\n\n"
                await asyncio.sleep(0.1)
            for i, rejected in enumerate(rejected_images):
                stream_data = {
                    "type": "rejected_frame",
                    "rejected_index": i + 1,
                    "total_rejected": rejected_count,
                    "result": rejected
                }
                yield f"data: {json.dumps(stream_data)}\n\n"
                await asyncio.sleep(0.1)
            total_original_size = sum([img["size_bytes"] for img in all_images])
            total_rejected_size = sum([img["size_bytes"] for img in rejected_images])
            summary_data = {
                "type": "summary",
                "content": f"""🔬 **Advanced Development Processing with Frame Rejection (NO Cropping)**

📊 **Frame Statistics:**
- ✅ Total frames accepted: {frame_count}
- ❌ Total frames rejected: {rejected_count}
- 📈 Acceptance rate: {(frame_count/(frame_count+rejected_count)*100):.1f}%
- ✂️ Header/footer cropping: ❌ Disabled

📁 **Data Size:**
- Accepted frames: {total_original_size:,} bytes
- Rejected frames: {total_rejected_size:,} bytes
- Total processed: {(total_original_size + total_rejected_size):,} bytes

📍 **Session Directory Structure:**
   - Main session: {session_dir}
   - ✅ Before homography: {before_dir}
   - ✅ After homography: {after_dir}
   - ❌ **Rejected frames: {rejected_dir}**
   - 🔍 **Debug visualization: {debug_dir}**

🖼️ **Images in Memory:**
- Accepted images: {len(all_images)}
- Each image contains: PIL Image objects, metadata
- Ready for testing and development

🔧 **Enhanced Processing Flow:**
1. Read file data from form ✅
2. Create PIL Image directly from bytes ✅
3. Apply document scanner homography with quality checks ✅
4. **🔍 Quality validation (area, aspect ratio, corner angles) ✅**
5. **📊 Post-warp validation (size, distortion, content) ✅**
6. **❌ Reject poor quality transforms ✅**
7. **🔍 Save debug visualization images ✅**
8. Add to organized directory structure ✅

📂 **Directory Organization:**
development/
└── {timestamp}/
    ├── before_homography/ (original only)
    ├── after_homography/ (accepted)
    ├── **rejected_homography/** ❌
    └── **debug_visualization/**
        └── **frame_X/**
            ├── **01_grayscale.png**
            ├── **02_blurred.png**
            ├── **03_edges.png**
            ├── **04_contours_detected.png** (with analysis!)
            ├── **05_warped_result.png** (final result)
            ├── **success_info.txt** (with info)
            ├── **failure_reason.txt** (if failed)
            └── **contour_info.txt**

🔍 **Quality Checks Implemented:**
- ✅ **Pre-warp validation:**
  - Minimum contour area (10% of image)
  - Reasonable aspect ratio (< 5:1)
  - Corner angles close to 90° (±45°)
- ✅ **Post-warp validation:**
  - Minimum warped size (200px)
  - Maximum aspect ratio (< 10:1)
  - Content density (> 30% non-black pixels)

❌ **Rejection Reasons Tracked:**
- contour_too_small_area_X_min_Y
- extreme_aspect_ratio_X
- bad_corner_angles_[list]
- warped_too_small_WxH
- warped_extreme_aspect_X
- warped_mostly_empty_X%
- no_4_point_contour_found
- error_[exception_details]

💡 **Debug Features:**
- **Check {debug_dir} for detailed analysis!**
- Each frame has complete processing visualization
- Success/failure reasons clearly documented
- Quality metrics saved for analysis
- Rejected frames preserved for inspection

🚀 **Production-ready homography pipeline with quality control!**""",
                "total_original_size": total_original_size,
                "total_rejected_size": total_rejected_size,
                "accepted_images_in_memory": len(all_images),
                "rejected_images_count": len(rejected_images),
                "processing_results": processing_results,
                "rejected_results": rejected_images,
                "quality_metrics": {
                    "acceptance_rate": f"{(frame_count/(frame_count+rejected_count)*100):.1f}%" if (frame_count+rejected_count) > 0 else "N/A",
                    "total_processed": frame_count + rejected_count,
                    "accepted": frame_count,
                    "rejected": rejected_count
                },
                "cropping_info": {
                    "enabled": False,
                    "top_crop_px": 0,
                    "bottom_crop_px": 0,
                    "purpose": "No cropping applied"
                },
                "debug_info": {
                    "debug_directory": debug_dir,
                    "rejected_directory": rejected_dir,
                    "visualization_files": [
                        "01_grayscale.png - Original converted to grayscale",
                        "02_blurred.png - Gaussian blur applied",
                        "03_edges.png - Canny edge detection result",
                        "04_contours_detected.png - ALL CONTOURS WITH QUALITY ANALYSIS!",
                        "05_warped_result.png - Homography result",
                        "success_info.txt - Quality metrics",
                        "failure_reason.txt - Detailed rejection analysis",
                        "contour_info.txt - Complete contour analysis"
                    ]
                }
            }
            yield f"data: {json.dumps(summary_data)}\n\n"
            yield "data: [DONE]\n\n"
        return StreamingResponse(
            generate_development_stream(),
            media_type="text/event-stream",
            headers={
                "Cache-Control": "no-cache",
                "Connection": "keep-alive",
                "Content-Type": "text/event-stream",
                "Access-Control-Allow-Origin": "*",
                "Access-Control-Allow-Methods": "*",
                "Access-Control-Allow-Headers": "*"
            }
        )
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()
        return JSONResponse({
            "success": False,
            "error": f"Development processing failed: {str(e)}"
        })

async def start_development_server():
    import uvicorn
    try:
        print("🚀 Starting ENHANCED DEVELOPMENT server on http://localhost:8000")
        print("📁 Images organized: before_homography → after_homography")
        print("🔧 Document scanner homography with QUALITY CONTROL!")
        print("❌ Frame rejection for poor quality transforms!")
        print("🔍 DEBUG VISUALIZATION with quality metrics!")
        print("📂 Directory structure: development/TIMESTAMP/[before|after|rejected|debug]/")
        config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
        server = uvicorn.Server(config)
        await server.serve()
    except Exception as e:
        print(f"❌ Server error: {e}")

await start_development_server()

🚀 Starting ENHANCED DEVELOPMENT server on http://localhost:8000
📁 Images organized: before_homography → after_homography
🔧 Document scanner homography with QUALITY CONTROL!
❌ Frame rejection for poor quality transforms!
🔍 DEBUG VISUALIZATION with quality metrics!
📂 Directory structure: development/TIMESTAMP/[before|after|rejected|debug]/


INFO:     Started server process [28020]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


🔄 DEVELOPMENT FRAME PROCESSING WITH HOMOGRAPHY & PANORAMA & REJECTION (NO CROPPING)...
🔄 DEVELOPMENT FRAME PROCESSING WITH HOMOGRAPHY & PANORAMA & REJECTION (NO CROPPING)...
📥 Form data items: 14
📥 Form data items: 14
📝 Processing frame_0: 1401895 bytes
✅ Processing frame_0
🔍 Found 5 contours for frame_0
  Contour 0: Area=950787, Perimeter=4120, Points=4
  ✅ Found valid 4-point contour at index 0!
    Area: 950787, Aspect ratio: 1.78, Angles: ['90.3°', '87.7°', '90.5°', '91.4°']
✅ Applying homography transformation for frame_0
✅ ACCEPTED frame_0 - homography successful
📝 Processing frame_0: 1401895 bytes
✅ Processing frame_0
🔍 Found 5 contours for frame_0
  Contour 0: Area=950787, Perimeter=4120, Points=4
  ✅ Found valid 4-point contour at index 0!
    Area: 950787, Aspect ratio: 1.78, Angles: ['90.3°', '87.7°', '90.5°', '91.4°']
✅ Applying homography transformation for frame_0
✅ ACCEPTED frame_0 - homography successful
📝 Processing frame_1: 1548528 bytes
✅ Processing frame_1
🔍 Found 5

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [28020]


ADDING BLUR

In [ ]:
import nest_asyncio
nest_asyncio.apply()


from fastapi import FastAPI, Request
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse, StreamingResponse
import json
import asyncio
import io
import os
from datetime import datetime
import time
from PIL import Image
import cv2
import numpy as np

app = FastAPI()

# Enable CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

class ImageProcessor:
    @staticmethod
    def order_points(pts):
        """Order points in the order: top-left, top-right, bottom-right, bottom-left"""
        rect = np.zeros((4, 2), dtype="float32")
        s = pts.sum(axis=1)
        diff = np.diff(pts, axis=1)
        rect[0] = pts[np.argmin(s)]      # top-left
        rect[2] = pts[np.argmax(s)]      # bottom-right
        rect[1] = pts[np.argmin(diff)]   # top-right
        rect[3] = pts[np.argmax(diff)]   # bottom-left
        return rect

    @staticmethod
    def four_point_transform(image, pts):
        """Apply perspective transform to get bird's eye view"""
        rect = ImageProcessor.order_points(pts)
        (tl, tr, br, bl) = rect
        widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
        widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
        maxWidth = max(int(widthA), int(widthB))
        heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
        heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
        maxHeight = max(int(heightA), int(heightB))
        dst = np.array([
            [0, 0],
            [maxWidth - 1, 0],
            [maxWidth - 1, maxHeight - 1],
            [0, maxHeight - 1]
        ], dtype="float32")
        M = cv2.getPerspectiveTransform(rect, dst)
        warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
        return warped

    @staticmethod
    def document_scanner_homography(img, save_debug=True, debug_dir=None, frame_key="unknown", blur_threshold=100.0):
        """Apply document scanner homography transformation with quality checks, rejection, and blur detection"""
        try:
            img_cv = np.array(img)
            if len(img_cv.shape) == 3:
                img_cv = cv2.cvtColor(img_cv, cv2.COLOR_RGB2BGR)
            original = img_cv.copy()
            gray = cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY)
            blurred = cv2.GaussianBlur(gray, (5, 5), 0)
            edged = cv2.Canny(blurred, 75, 200)
            if save_debug and debug_dir:
                debug_frame_dir = os.path.join(debug_dir, frame_key)
                os.makedirs(debug_frame_dir, exist_ok=True)
                cv2.imwrite(os.path.join(debug_frame_dir, "01_grayscale.png"), gray)
                cv2.imwrite(os.path.join(debug_frame_dir, "02_blurred.png"), blurred)
                cv2.imwrite(os.path.join(debug_frame_dir, "03_edges.png"), edged)
            contours, _ = cv2.findContours(edged, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
            contours = sorted(contours, key=cv2.contourArea, reverse=True)[:5]
            print(f"🔍 Found {len(contours)} contours for {frame_key}")
            debug_contours = original.copy()
            cv2.drawContours(debug_contours, contours, -1, (0, 255, 0), 2)
            screenCnt = None
            contour_info = []
            rejection_reason = None
            for idx, c in enumerate(contours):
                area = cv2.contourArea(c)
                peri = cv2.arcLength(c, True)
                approx = cv2.approxPolyDP(c, 0.02 * peri, True)
                contour_info.append({
                    "index": idx,
                    "area": area,
                    "perimeter": peri,
                    "points": len(approx)
                })
                print(f"  Contour {idx}: Area={area:.0f}, Perimeter={peri:.0f}, Points={len(approx)}")
                if len(approx) >= 3:
                    M = cv2.moments(c)
                    if M["m00"] != 0:
                        cx = int(M["m10"] / M["m00"])
                        cy = int(M["m01"] / M["m00"])
                        cv2.putText(debug_contours, f"{idx}({len(approx)}p)", (cx-20, cy),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
                if len(approx) == 4:
                    min_area = (img_cv.shape[0] * img_cv.shape[1]) * 0.1
                    if area < min_area:
                        rejection_reason = f"contour_too_small_area_{area:.0f}_min_{min_area:.0f}"
                        print(f"  ❌ Contour {idx} rejected: too small (area: {area:.0f} < {min_area:.0f})")
                        continue
                    rect = ImageProcessor.order_points(approx.reshape(4, 2))
                    (tl, tr, br, bl) = rect
                    width1 = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
                    width2 = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
                    height1 = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
                    height2 = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
                    avg_width = (width1 + width2) / 2
                    avg_height = (height1 + height2) / 2
                    aspect_ratio = max(avg_width, avg_height) / min(avg_width, avg_height)
                    if aspect_ratio > 5.0:
                        rejection_reason = f"extreme_aspect_ratio_{aspect_ratio:.2f}"
                        print(f"  ❌ Contour {idx} rejected: extreme aspect ratio ({aspect_ratio:.2f})")
                        continue
                    def angle_between_vectors(v1, v2):
                        cos_angle = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
                        cos_angle = np.clip(cos_angle, -1.0, 1.0)
                        return np.degrees(np.arccos(cos_angle))
                    corners = approx.reshape(4, 2)
                    angles = []
                    for i in range(4):
                        p1 = corners[i]
                        p2 = corners[(i + 1) % 4]
                        p3 = corners[(i + 2) % 4]
                        v1 = p1 - p2
                        v2 = p3 - p2
                        angle = angle_between_vectors(v1, v2)
                        angles.append(angle)
                    angle_threshold = 45
                    bad_angles = [abs(angle - 90) for angle in angles if abs(angle - 90) > angle_threshold]
                    if bad_angles:
                        rejection_reason = f"bad_corner_angles_{bad_angles}"
                        print(f"  ❌ Contour {idx} rejected: bad corner angles {angles}")
                        continue
                    print(f"  ✅ Found valid 4-point contour at index {idx}!")
                    print(f"    Area: {area:.0f}, Aspect ratio: {aspect_ratio:.2f}, Angles: {[f'{a:.1f}°' for a in angles]}")
                    screenCnt = approx
                    cv2.drawContours(debug_contours, [approx], -1, (0, 0, 255), 3)
                    for i, point in enumerate(approx):
                        cv2.circle(debug_contours, tuple(point[0]), 8, (255, 255, 0), -1)
                        cv2.putText(debug_contours, str(i), tuple(point[0] + 10),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
                    break
            if save_debug and debug_dir:
                cv2.imwrite(os.path.join(debug_frame_dir, "04_contours_detected.png"), debug_contours)
                with open(os.path.join(debug_frame_dir, "contour_info.txt"), "w") as f:
                    f.write(f"Contour Analysis for {frame_key}\n")
                    f.write("=" * 40 + "\n")
                    for info in contour_info:
                        f.write(f"Contour {info['index']}: Area={info['area']:.0f}, "
                                f"Perimeter={info['perimeter']:.0f}, Points={info['points']}\n")
                    f.write(f"\nSelected contour: {'Found' if screenCnt is not None else 'None'}\n")
                    if rejection_reason:
                        f.write(f"Rejection reason: {rejection_reason}\n")
            if screenCnt is not None:
                print(f"✅ Applying homography transformation for {frame_key}")
                warped = ImageProcessor.four_point_transform(original, screenCnt.reshape(4, 2))
                min_warped_size = 200
                if warped.shape[0] < min_warped_size or warped.shape[1] < min_warped_size:
                    rejection_reason = f"warped_too_small_{warped.shape[1]}x{warped.shape[0]}"
                    print(f"  ❌ Warped result rejected: too small ({warped.shape[1]}x{warped.shape[0]})")
                    raise ValueError(f"Warped image too small: {warped.shape}")
                warp_aspect = max(warped.shape[0], warped.shape[1]) / min(warped.shape[0], warped.shape[1])
                if warp_aspect > 10.0:
                    rejection_reason = f"warped_extreme_aspect_{warp_aspect:.2f}"
                    print(f"  ❌ Warped result rejected: extreme aspect ratio ({warp_aspect:.2f})")
                    raise ValueError(f"Warped image too distorted: aspect ratio {warp_aspect:.2f}")
                gray_warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
                non_zero_pixels = np.count_nonzero(gray_warped > 50)
                total_pixels = gray_warped.shape[0] * gray_warped.shape[1]
                content_ratio = non_zero_pixels / total_pixels
                if content_ratio < 0.3:
                    rejection_reason = f"warped_mostly_empty_{content_ratio:.2f}"
                    print(f"  ❌ Warped result rejected: mostly empty ({content_ratio:.2%} content)")
                    raise ValueError(f"Warped image mostly empty: {content_ratio:.2%} content")
                # Blur detection using variance of Laplacian
                laplacian_var = cv2.Laplacian(gray_warped, cv2.CV_64F).var()
                if laplacian_var < blur_threshold:
                    rejection_reason = f"warped_blurry_var_{laplacian_var:.1f}_thresh_{blur_threshold}"
                    print(f"  ❌ Warped result rejected: blurry (Laplacian variance {laplacian_var:.1f} < {blur_threshold})")
                    if save_debug and debug_dir:
                        with open(os.path.join(debug_frame_dir, "failure_reason.txt"), "a") as f:
                            f.write(f"Blur detection: Laplacian variance {laplacian_var:.1f} < {blur_threshold}\n")
                    raise ValueError(f"Warped image too blurry: Laplacian variance {laplacian_var:.1f}")
                if save_debug and debug_dir:
                    cv2.imwrite(os.path.join(debug_frame_dir, "05_warped_result.png"), warped)
                    with open(os.path.join(debug_frame_dir, "success_info.txt"), "w") as f:
                        f.write(f"Homography successful for {frame_key}\n")
                        f.write(f"Warped size: {warped.shape[1]}x{warped.shape[0]}\n")
                        f.write(f"Warped aspect ratio: {warp_aspect:.2f}\n")
                        f.write(f"Content ratio: {content_ratio:.2%}\n")
                        f.write(f"Blur (Laplacian variance): {laplacian_var:.1f}\n")
                if len(warped.shape) == 3:
                    warped = cv2.cvtColor(warped, cv2.COLOR_BGR2RGB)
                return Image.fromarray(warped), "success"
            else:
                if not rejection_reason:
                    rejection_reason = "no_4_point_contour_found"
                print(f"⚠️ No valid 4-point contour found for {frame_key}")
                if save_debug and debug_dir:
                    with open(os.path.join(debug_frame_dir, "failure_reason.txt"), "w") as f:
                        f.write(f"Homography failed for {frame_key}\n")
                        f.write(f"Reason: {rejection_reason}\n\n")
                        f.write("Suggestions:\n")
                        f.write("1. Check if image has clear document edges\n")
                        f.write("2. Adjust Canny edge detection parameters\n")
                        f.write("3. Try different contour approximation epsilon\n")
                        f.write("4. Ensure good contrast between document and background\n")
                        f.write("5. Make sure document occupies significant portion of image\n")
                return None, rejection_reason
        except Exception as e:
            error_msg = f"error_{str(e).replace(' ', '_')}"
            print(f"❌ Homography error for {frame_key}: {e}")
            if save_debug and debug_dir:
                debug_frame_dir = os.path.join(debug_dir, frame_key)
                os.makedirs(debug_frame_dir, exist_ok=True)
                with open(os.path.join(debug_frame_dir, "error.txt"), "w") as f:
                    f.write(f"Error processing {frame_key}: {str(e)}\n")
            return None, error_msg

@app.post("/process-multiple-frames-stream")
async def process_frames_development(request: Request):
    """Development endpoint with homography and panorama functionality with frame rejection (NO cropping)"""
    try:
        print("=" * 50)
        print("🔄 DEVELOPMENT FRAME PROCESSING WITH HOMOGRAPHY & PANORAMA & REJECTION (NO CROPPING)...")
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        base_dir = "development"
        session_dir = os.path.join(base_dir, timestamp)
        before_dir = os.path.join(session_dir, "before_homography")
        after_dir = os.path.join(session_dir, "after_homography")
        rejected_dir = os.path.join(session_dir, "rejected_homography")
        debug_dir = os.path.join(session_dir, "debug_visualization")
        os.makedirs(before_dir, exist_ok=True)
        os.makedirs(after_dir, exist_ok=True)
        os.makedirs(rejected_dir, exist_ok=True)
        os.makedirs(debug_dir, exist_ok=True)
        form_data = await request.form()
        print(f"📥 Form data items: {len(form_data)}")
        all_images = []
        rejected_images = []
        processing_results = []
        frame_count = 0
        rejected_count = 0
        for key, value in form_data.items():
            if key.startswith('frame_') and hasattr(value, 'read'):
                try:
                    file_data = await value.read()
                    print(f"📝 Processing {key}: {len(file_data)} bytes")
                    original_img = Image.open(io.BytesIO(file_data))
                    print(f"✅ Processing {key}")
                    processed_img, status = ImageProcessor.document_scanner_homography(
                        original_img,
                        save_debug=True,
                        debug_dir=debug_dir,
                        frame_key=key,
                        blur_threshold=100.0  # You can adjust this threshold
                    )
                    if processed_img is not None and status == "success":
                        image_data = {
                            "key": key,
                            "original_image": original_img,
                            "processed_image": processed_img,
                            "file_data": file_data,
                            "filename": getattr(value, 'filename', 'unknown'),
                            "size_bytes": len(file_data),
                            "image_size": f"{original_img.width}x{original_img.height}",
                            "processed_size": f"{processed_img.width}x{processed_img.height}",
                            "status": "accepted"
                        }
                        all_images.append(image_data)
                        before_path = os.path.join(before_dir, f"{key}_original.png")
                        after_path = os.path.join(after_dir, f"{key}_homography.png")
                        original_img.save(before_path)
                        processed_img.save(after_path)
                        processing_results.append({
                            "frame_key": key,
                            "original_path": before_path,
                            "processed_path": after_path,
                            "debug_path": os.path.join(debug_dir, key),
                            "original_size": f"{original_img.width}x{original_img.height}",
                            "processed_size": f"{processed_img.width}x{processed_img.height}",
                            "original_file_size": len(file_data),
                            "processed_file_size": os.path.getsize(after_path),
                            "status": "accepted"
                        })
                        frame_count += 1
                        print(f"✅ ACCEPTED {key} - homography successful")
                    else:
                        rejected_path = os.path.join(rejected_dir, f"{key}_rejected_{status}.png")
                        original_img.save(rejected_path)
                        rejected_images.append({
                            "key": key,
                            "filename": getattr(value, 'filename', 'unknown'),
                            "size_bytes": len(file_data),
                            "image_size": f"{original_img.width}x{original_img.height}",
                            "rejection_reason": status,
                            "rejected_path": rejected_path,
                            "debug_path": os.path.join(debug_dir, key)
                        })
                        rejected_count += 1
                        print(f"❌ REJECTED {key} - {status}")
                except Exception as frame_error:
                    print(f"❌ Error processing {key}: {frame_error}")
        print(f"🎉 Successfully processed {frame_count} frames!")
        print(f"❌ Rejected {rejected_count} frames!")
        print(f"📋 {len(all_images)} accepted images loaded into list for testing")
        async def generate_development_stream():
            initial_response = {
                "success": True,
                "message": f"Development processing complete! {frame_count} accepted, {rejected_count} rejected (NO cropping)",
                "timestamp": timestamp,
                "session_directory": session_dir,
                "directories": {
                    "session": session_dir,
                    "before_homography": before_dir,
                    "after_homography": after_dir,
                    "rejected_homography": rejected_dir,
                    "debug_visualization": debug_dir
                },
                "frame_count": frame_count,
                "rejected_count": rejected_count,
                "all_images_count": len(all_images),
                "header_footer_cropped": False,
                "type": "initial"
            }
            yield f"data: {json.dumps(initial_response)}\n\n"
            await asyncio.sleep(0.3)
            for i, result in enumerate(processing_results):
                stream_data = {
                    "type": "frame_result",
                    "frame_index": i + 1,
                    "total_frames": frame_count,
                    "result": result
                }
                yield f"data: {json.dumps(stream_data)}\n\n"
                await asyncio.sleep(0.1)
            for i, rejected in enumerate(rejected_images):
                stream_data = {
                    "type": "rejected_frame",
                    "rejected_index": i + 1,
                    "total_rejected": rejected_count,
                    "result": rejected
                }
                yield f"data: {json.dumps(stream_data)}\n\n"
                await asyncio.sleep(0.1)
            total_original_size = sum([img["size_bytes"] for img in all_images])
            total_rejected_size = sum([img["size_bytes"] for img in rejected_images])
            summary_data = {
                "type": "summary",
                "content": f"""🔬 **Advanced Development Processing with Frame Rejection (NO Cropping)**

📊 **Frame Statistics:**
- ✅ Total frames accepted: {frame_count}
- ❌ Total frames rejected: {rejected_count}
- 📈 Acceptance rate: {(frame_count/(frame_count+rejected_count)*100):.1f}%
- ✂️ Header/footer cropping: ❌ Disabled

📁 **Data Size:**
- Accepted frames: {total_original_size:,} bytes
- Rejected frames: {total_rejected_size:,} bytes
- Total processed: {(total_original_size + total_rejected_size):,} bytes

📍 **Session Directory Structure:**
   - Main session: {session_dir}
   - ✅ Before homography: {before_dir}
   - ✅ After homography: {after_dir}
   - ❌ **Rejected frames: {rejected_dir}**
   - 🔍 **Debug visualization: {debug_dir}**

🖼️ **Images in Memory:**
- Accepted images: {len(all_images)}
- Each image contains: PIL Image objects, metadata
- Ready for testing and development

🔧 **Enhanced Processing Flow:**
1. Read file data from form ✅
2. Create PIL Image directly from bytes ✅
3. Apply document scanner homography with quality checks ✅
4. **🔍 Quality validation (area, aspect ratio, corner angles) ✅**
5. **📊 Post-warp validation (size, distortion, content) ✅**
6. **❌ Reject poor quality transforms ✅**
7. **🔍 Save debug visualization images ✅**
8. Add to organized directory structure ✅

📂 **Directory Organization:**
development/
└── {timestamp}/
    ├── before_homography/ (original only)
    ├── after_homography/ (accepted)
    ├── **rejected_homography/** ❌
    └── **debug_visualization/**
        └── **frame_X/**
            ├── **01_grayscale.png**
            ├── **02_blurred.png**
            ├── **03_edges.png**
            ├── **04_contours_detected.png** (with analysis!)
            ├── **05_warped_result.png** (final result)
            ├── **success_info.txt** (with info)
            ├── **failure_reason.txt** (if failed)
            └── **contour_info.txt**

🔍 **Quality Checks Implemented:**
- ✅ **Pre-warp validation:**
  - Minimum contour area (10% of image)
  - Reasonable aspect ratio (< 5:1)
  - Corner angles close to 90° (±45°)
- ✅ **Post-warp validation:**
  - Minimum warped size (200px)
  - Maximum aspect ratio (< 10:1)
  - Content density (> 30% non-black pixels)
  - **Blur detection (Laplacian variance > 100.0)**

❌ **Rejection Reasons Tracked:**
- contour_too_small_area_X_min_Y
- extreme_aspect_ratio_X
- bad_corner_angles_[list]
- warped_too_small_WxH
- warped_extreme_aspect_X
- warped_mostly_empty_X%
- warped_blurry_var_X_thresh_Y
- no_4_point_contour_found
- error_[exception_details]

💡 **Debug Features:**
- **Check {debug_dir} for detailed analysis!**
- Each frame has complete processing visualization
- Success/failure reasons clearly documented
- Quality metrics saved for analysis
- Rejected frames preserved for inspection

🚀 **Production-ready homography pipeline with quality control!**""",
                "total_original_size": total_original_size,
                "total_rejected_size": total_rejected_size,
                "accepted_images_in_memory": len(all_images),
                "rejected_images_count": len(rejected_images),
                "processing_results": processing_results,
                "rejected_results": rejected_images,
                "quality_metrics": {
                    "acceptance_rate": f"{(frame_count/(frame_count+rejected_count)*100):.1f}%" if (frame_count+rejected_count) > 0 else "N/A",
                    "total_processed": frame_count + rejected_count,
                    "accepted": frame_count,
                    "rejected": rejected_count
                },
                "cropping_info": {
                    "enabled": False,
                    "top_crop_px": 0,
                    "bottom_crop_px": 0,
                    "purpose": "No cropping applied"
                },
                "debug_info": {
                    "debug_directory": debug_dir,
                    "rejected_directory": rejected_dir,
                    "visualization_files": [
                        "01_grayscale.png - Original converted to grayscale",
                        "02_blurred.png - Gaussian blur applied",
                        "03_edges.png - Canny edge detection result",
                        "04_contours_detected.png - ALL CONTOURS WITH QUALITY ANALYSIS!",
                        "05_warped_result.png - Homography result",
                        "success_info.txt - Quality metrics",
                        "failure_reason.txt - Detailed rejection analysis",
                        "contour_info.txt - Complete contour analysis"
                    ]
                }
            }
            yield f"data: {json.dumps(summary_data)}\n\n"
            yield "data: [DONE]\n\n"
        return StreamingResponse(
            generate_development_stream(),
            media_type="text/event-stream",
            headers={
                "Cache-Control": "no-cache",
                "Connection": "keep-alive",
                "Content-Type": "text/event-stream",
                "Access-Control-Allow-Origin": "*",
                "Access-Control-Allow-Methods": "*",
                "Access-Control-Allow-Headers": "*"
            }
        )
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()
        return JSONResponse({
            "success": False,
            "error": f"Development processing failed: {str(e)}"
        })

async def start_development_server():
    import uvicorn
    try:
        print("🚀 Starting ENHANCED DEVELOPMENT server on http://localhost:8000")
        print("📁 Images organized: before_homography → after_homography")
        print("🔧 Document scanner homography with QUALITY CONTROL!")
        print("❌ Frame rejection for poor quality transforms!")
        print("🔍 DEBUG VISUALIZATION with quality metrics!")
        print("📂 Directory structure: development/TIMESTAMP/[before|after|rejected|debug]/")
        config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
        server = uvicorn.Server(config)
        await server.serve()
    except Exception as e:
        print(f"❌ Server error: {e}")

await start_development_server()

🚀 Starting ENHANCED DEVELOPMENT server on http://localhost:8000
📁 Images organized: before_homography → after_homography
🔧 Document scanner homography with QUALITY CONTROL!
❌ Frame rejection for poor quality transforms!
🔍 DEBUG VISUALIZATION with quality metrics!
📂 Directory structure: development/TIMESTAMP/[before|after|rejected|debug]/


INFO:     Started server process [28020]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


🔄 DEVELOPMENT FRAME PROCESSING WITH HOMOGRAPHY & PANORAMA & REJECTION (NO CROPPING)...
🔄 DEVELOPMENT FRAME PROCESSING WITH HOMOGRAPHY & PANORAMA & REJECTION (NO CROPPING)...
📥 Form data items: 14
📝 Processing frame_0: 1408380 bytes
✅ Processing frame_0
🔍 Found 5 contours for frame_0
  Contour 0: Area=899882, Perimeter=4109, Points=4
  ✅ Found valid 4-point contour at index 0!
    Area: 899882, Aspect ratio: 1.78, Angles: ['86.7°', '88.9°', '93.1°', '91.2°']
✅ Applying homography transformation for frame_0
✅ ACCEPTED frame_0 - homography successful
📥 Form data items: 14
📝 Processing frame_1: 1319865 bytes
✅ Processing frame_1
🔍 Found 5 contours for frame_1
  Contour 0: Area=902575, Perimeter=4125, Points=4
  ✅ Found valid 4-point contour at index 0!
    Area: 902575, Aspect ratio: 1.78, Angles: ['86.8°', '89.2°', '93.0°', '91.0°']
✅ Applying homography transformation for frame_1
✅ ACCEPTED frame_1 - homography successful
📝 Processing frame_0: 1408380 bytes
✅ Processing frame_0
🔍 Found 5

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [28020]
